# Anticipez les besoins en consommation de bâtiments - *Notebook prediction SiteEnergyUseWN(kBtu)*

## Mission

Vous travaillez pour la ville de Seattle. Pour atteindre son objectif de ville neutre en émissions de carbone en 2050, votre équipe s’intéresse de près à la consommation et aux émissions des bâtiments non destinés à l’habitation.

Des relevés minutieux ont été effectués par les agents de la ville en 2016. Cependant, ces relevés sont coûteux à obtenir, et à partir de ceux déjà réalisés, vous voulez tenter de prédire les émissions de CO2 et la consommation totale d’énergie de bâtiments non destinés à l’habitation pour lesquels elles n’ont pas encore été mesurées.

Votre prédiction se basera sur les données structurelles des bâtiments (taille et usage des bâtiments, date de construction, situation géographique, ...)

Vous cherchez également à évaluer l’intérêt de l’ENERGY STAR Score pour la prédiction d’émissions, qui est fastidieux à calculer avec l’approche utilisée actuellement par votre équipe. Vous l'intégrerez dans la modélisation et jugerez de son intérêt.

Vous sortez tout juste d’une réunion de brief avec votre équipe. Voici un récapitulatif de votre mission :


1) Réaliser une courte analyse exploratoire.
2) Tester différents modèles de prédiction afin de répondre au mieux à la problématique.

Fais bien attention au traitement des différentes variables, à la fois pour trouver de nouvelles informations (peut-on déduire des choses intéressantes d’une simple adresse ?) et optimiser les performances en appliquant des transformations simples aux variables (normalisation, passage au log, etc.).

Mets en place une évaluation rigoureuse des performances, et optimise les hyperparamètres et le choix d’algorithmes de ML à l’aide d’une validation croisée. Tu testeras au minimum 4 algorithmes de famille différente (par exemple : ElasticNet, SVM, GradientBoosting, RandomForest).

In [4]:
import numpy as np

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import FuncFormatter
import scipy
from scipy import stats
import scipy.stats as st

import statsmodels
import statsmodels.api as sm
import missingno as msno

import sklearn
from sklearn.experimental import enable_iterative_imputer  # Nécessaire pour activer IterativeImputer
from sklearn.impute import IterativeImputer

from sklearn.impute import KNNImputer
# Encodage des variables catégorielles avant d'utiliser KNNImputer
from category_encoders.ordinal import OrdinalEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

# pour le centrage et la réduction
from sklearn.preprocessing import StandardScaler
# pour l'ACP
from sklearn.decomposition import PCA

from sklearn import model_selection
from sklearn.model_selection import GridSearchCV

from sklearn import metrics
from sklearn.metrics import roc_curve, auc, confusion_matrix, mean_squared_error, make_scorer, r2_score, mean_absolute_error

from sklearn import dummy
from sklearn.dummy import DummyClassifier
from sklearn.dummy import DummyRegressor

from sklearn import linear_model
from sklearn.linear_model import Ridge
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression

from sklearn.svm import LinearSVC
from sklearn.svm import SVR

from sklearn import kernel_ridge

from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier

import tensorflow
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers

from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from xgboost import XGBRegressor

import shap

shap.initjs()

import timeit
import warnings

print("numpy version", np.__version__)
print("pandas version", pd.__version__)
print("matplotlib version", matplotlib.__version__)
print("seaborn version", sns.__version__)
print("scipy version", scipy.__version__)
print("statsmodels version", statsmodels.__version__)
print("missingno version", msno.__version__)

print("sklearn version", sklearn.__version__)
print("tensorflow version", tensorflow.__version__)
print("shap  version", shap.__version__)

pd.options.display.max_rows = 200
pd.options.display.max_columns = 100

numpy version 1.26.4
pandas version 2.1.4
matplotlib version 3.8.0
seaborn version 0.13.2
scipy version 1.11.4
statsmodels version 0.14.0
missingno version 0.5.2
sklearn version 1.2.2
tensorflow version 2.18.0


## 1 - Création des fonctions et paramètres pour automatisation

### 1.1 - Fonction des modèles sans validation croisée

**Baseline avec DummyRegressor**

On va utiliser la stratégie de la moyenne : prédit la moyenne des valeurs cibles d'entraînement. Créons la fonction qui prend les jeux d'entraînement et de tests et en entrée, et retourne les scores MSE, RMSE, R2, MAE:

In [8]:
def fit_dummyRegressor(X_train, y_train, X_test, y_test):

    start_time = timeit.default_timer()
    
    # Initialisation du DummyRegressor avec la stratégie 'mean'
    dummy_regressor = DummyRegressor(strategy='mean')
    
    # Entraînement du modèle
    dummy_regressor.fit(X_train, y_train)
    
    # Prédiction sur les données de test
    y_pred = dummy_regressor.predict(X_test)

    elapsed = round(timeit.default_timer() - start_time, 3)
    
    mse = round(mean_squared_error(y_test, y_pred), 2)       # Erreur quadratique moyenne
    rmse = round(np.sqrt(mse), 2)                            # Racine carrée de l'erreur quadratique moyenne (RMSE)
    mae = round(mean_absolute_error(y_test, y_pred), 2)      # Erreur absolue moyenne
    r2 = round(r2_score(y_test, y_pred), 2)                  # Coefficient de détermination

    return mse, rmse, r2, mae, elapsed

**Modèle de régression Ridge**

La régression ridge nous permet de réduire l'amplitude des coefficients d'une régression linéaire et d'éviter le sur-apprentissage. On optimisera l'hyperparamètre alpha lors de la validation croisée avec GridSearchCV. Pour rappel, dans le cadre de la régression Ridge, l'hyperparamètre alpha contrôle le degré de régularisation L2 appliqué au modèle. Il agit comme un paramètre de pénalisation pour limiter la taille des coefficients du modèle afin de réduire le sur-ajustement.

Créons une fonction qui permet d'instancier le modèle, l'entraîner, et calculer les scores:

In [10]:
def fit_ridge(X_train, y_train, X_test, y_test):

    start_time = timeit.default_timer()
    
    # Initialisation du modèle Ridge avec un paramètre alpha
    ridge_regressor = Ridge(alpha=1)  # alpha contrôle la régularisation ; plus grand, plus de régularisation ( 1 = valeur par défaut)
    
    # Entraînement du modèle
    ridge_regressor.fit(X_train, y_train)
    
    # Prédiction sur les données de test
    y_pred = ridge_regressor.predict(X_test)

    elapsed = round(timeit.default_timer() - start_time, 3)
    
    # Évaluation du modèle avec différentes métriques
    mse = round(mean_squared_error(y_test, y_pred), 2)       # Erreur quadratique moyenne
    rmse = round(np.sqrt(mse), 2)                            # Racine carrée de l'erreur quadratique moyenne (RMSE)
    mae = round(mean_absolute_error(y_test, y_pred), 2)      # Erreur absolue moyenne
    r2 = round(r2_score(y_test, y_pred), 2)                  # Coefficient de détermination

    return mse, rmse, r2, mae, elapsed

**Modèle de régression Lasso**

Le Lasso est une méthode de sélection de variables et de réduction de dimension supervisée : les variables qui ne sont pas nécessaires à la prédiction de l'étiquette sont éliminées. l'hyperparamètre alpha contrôle l'intensité de la pénalisation L1 appliquée au modèle. Il prendre la valeur par défaut 1 ici.

Créons une fonction qui permet d'instancier le modèle, l'entraîner, et calculer les scores:

In [12]:
def fit_lasso(X_train, y_train, X_test, y_test):

    start_time = timeit.default_timer()
    
    # Initialisation du modèle Lasso avec un paramètre alpha
    lasso_regressor = Lasso(alpha=1)  # alpha contrôle la régularisation ; plus grand, plus de régularisation
    
    # Entraînement du modèle
    lasso_regressor.fit(X_train, y_train)
    
    # Prédiction sur les données de test
    y_pred = lasso_regressor.predict(X_test)

    elapsed = round(timeit.default_timer() - start_time, 3)
    
    # Évaluation du modèle avec différentes métriques
    mse = round(mean_squared_error(y_test, y_pred), 2)       # Erreur quadratique moyenne
    rmse = round(np.sqrt(mse), 2)                            # Racine carrée de l'erreur quadratique moyenne (RMSE)
    mae = round(mean_absolute_error(y_test, y_pred), 2)      # Erreur absolue moyenne
    r2 = round(r2_score(y_test, y_pred), 2)                  # Coefficient de détermination

    return mse, rmse, r2, mae, elapsed

**Modèle ElasticNet**

La méthode elastic net qui combine les deux termes de régularisation en un (Ridge L2 et Lasso L1). Deux hyperparamètres principaux sont à ajuster :
- alpha : contrôle la force globale de la régularisation (combine L1 et L2). (Par défaut = 1)
- l1_ratio : détermine la pondération entre la pénalisation L1 (type LASSO) et L2 (type Ridge). (Par défaut = 0.5)

Créons une fonction qui permet d'instancier le modèle, l'entraîner, et calculer les scores:

In [14]:
def fit_elasticNet(X_train, y_train, X_test, y_test):

    start_time = timeit.default_timer()

    # Initialisation du modèle ElasticNet
    elastic_net = ElasticNet(
        alpha=1, 
        l1_ratio=0.5, # Mélange égal entre ridge et lasso
        max_iter=1000, random_state=42
    )

    # Entraînement du modèle
    elastic_net.fit(X_train, y_train)

    # Prédictions sur le jeu de test
    y_pred = elastic_net.predict(X_test)
    
    elapsed = round(timeit.default_timer() - start_time, 3)
    
    # Évaluation du modèle avec différentes métriques
    mse = round(mean_squared_error(y_test, y_pred), 2)       # Erreur quadratique moyenne
    rmse = round(np.sqrt(mse), 2)                            # Racine carrée de l'erreur quadratique moyenne (RMSE)
    mae = round(mean_absolute_error(y_test, y_pred), 2)      # Erreur absolue moyenne
    r2 = round(r2_score(y_test, y_pred), 2)                  # Coefficient de détermination

    return mse, rmse, r2, mae, elapsed

**Bagging - Modèle RamdonForestRegressor**

Appelé aussi le bagging qui, appliqué aux arbres de décision, donne naissance au modèle de forêt aléatoire. 

Une forêt aléatoire est un ensemble de nombreux arbres de décision qui sont combinés pour produire une prédiction plus précise et plus robuste. Chaque arbre de décision est construit à partir d'un échantillon aléatoire des données et les résultats sont moyennés pour obtenir la prédiction finale. 

Le modèle de forêt aléatoire est intrinsèquement parallèle. Les arbres sont entraînés en même temps sur des parties du dataset.

Créons une fonction qui permet d'instancier le modèle, l'entraîner, et calculer les scores:

In [16]:
def fit_ramdomForestRegressor(X_train, y_train, X_test, y_test, data_columns):

    start_time = timeit.default_timer()
    
    # Création du modèle
    # n_estimators : Nombre d'arbres dans la forêt. Valeur par défaut = 100. Une valeur plus élevée peut améliorer la précision mais augmente le temps de calcul.
    model = RandomForestRegressor(n_estimators=100, random_state=42) 
    
    # Entraînement du modèle
    model.fit(X_train, y_train)
    
    # Prédictions
    y_pred = model.predict(X_test)

    elapsed = round(timeit.default_timer() - start_time, 3)
    
    # Évaluation du modèle avec différentes métriques
    mse = round(mean_squared_error(y_test, y_pred), 2)       # Erreur quadratique moyenne
    rmse = round(np.sqrt(mse), 2)                            # Racine carrée de l'erreur quadratique moyenne (RMSE)
    mae = round(mean_absolute_error(y_test, y_pred), 2)      # Erreur absolue moyenne
    r2 = round(r2_score(y_test, y_pred), 2)                  # Coefficient de détermination

    # Afficher l'importance des features
    print("Importance des features dans le RandomForestRegressor :")
    importances = model.feature_importances_
    # Création d'un DataFrame pour afficher l'importance des features
    feature_importance = pd.DataFrame({'Feature': data_columns, 'Importance': importances})
    # Tri par ordre d'importance décroissante
    feature_importance = feature_importance.sort_values(by='Importance', ascending=False)

    # Afficher les résultats
    print(feature_importance)

    return mse, rmse, r2, mae, elapsed

**Boosting - Modèle GradientBoostingRegressor**

Le boosting enchaîne l'entraînement des prédicteurs faibles de façon séquentielle, en se concentrant à chaque itération sur les échantillons qui ont généré le plus d'erreurs.

Il s'agit d'une méthode d'ensemble qui construit un modèle prédictif puissant en combinant plusieurs modèles faibles (typiquement des arbres de décision) de manière séquentielle

Le modèle de forêt aléatoire est intrinsèquement parallèle. Les arbres sont entraînés en même temps sur des parties du dataset.

Créons une fonction qui permet d'instancier le modèle, l'entraîner, et calculer les scores:

In [18]:
def fit_gradientBoostingRegressor(X_train, y_train, X_test, y_test):

    start_time = timeit.default_timer()

    # Création du modèle de régression Gradient Boosting
    gb_regressor = GradientBoostingRegressor(
        n_estimators=100,  # Nombre d'arbres
        learning_rate=0.1,  # Taux d'apprentissage (ici valeur par défaut)
        max_depth=3,  # Profondeur maximale des arbres (ici valeur par défaut)
        random_state=42  # Pour la reproductibilité
    )

    # Entraînement du modèle
    gb_regressor.fit(X_train, y_train)

    # Prédictions sur le jeu de test
    y_pred = gb_regressor.predict(X_test)

    elapsed = round(timeit.default_timer() - start_time, 3)
    
    # Évaluation du modèle avec différentes métriques
    mse = round(mean_squared_error(y_test, y_pred), 2)       # Erreur quadratique moyenne
    rmse = round(np.sqrt(mse), 2)                            # Racine carrée de l'erreur quadratique moyenne (RMSE)
    mae = round(mean_absolute_error(y_test, y_pred), 2)      # Erreur absolue moyenne
    r2 = round(r2_score(y_test, y_pred), 2)                  # Coefficient de détermination

    return mse, rmse, r2, mae, elapsed

**Modèle SVR**

Pour effectuer une régression avec un SVM (Support Vector Machine), on utilise le modèle appelé Support Vector Regression (SVR), qui fait partie des algorithmes de régression basés sur les SVM.

Le SVR cherche à trouver une fonction qui ne s'écarte pas trop des valeurs cibles, avec un contrôle sur la marge d'erreur autorisée.

Créons une fonction qui permet d'instancier le modèle, l'entraîner, et calculer les scores:

In [20]:
def fit_SVR(X_train, y_train, X_test, y_test):
    
    start_time = timeit.default_timer()
    
    # Initialisation du modèle SVR
    svr_model = SVR(
        kernel='rbf', 
        C=1.0,  # Paramètre de régularisation qui contrôle la pénalité pour les erreurs. Un C élevé cherche à minimiser les erreurs.
        epsilon=0.1 # Contrôle la largeur de la marge autour de la fonction cible. Les points situés dans cette marge ne contribuent pas à la fonction de coût
    )

    # Entraînement du modèle SVR
    svr_model.fit(X_train, y_train)

    # Prédictions sur le jeu de test
    y_pred = svr_model.predict(X_test)

    elapsed = round(timeit.default_timer() - start_time, 3)
    
    # Évaluation du modèle avec différentes métriques
    mse = round(mean_squared_error(y_test, y_pred), 2)       # Erreur quadratique moyenne
    rmse = round(np.sqrt(mse), 2)                            # Racine carrée de l'erreur quadratique moyenne (RMSE)
    mae = round(mean_absolute_error(y_test, y_pred), 2)      # Erreur absolue moyenne
    r2 = round(r2_score(y_test, y_pred), 2)                  # Coefficient de détermination

    return mse, rmse, r2, mae, elapsed

**Fonction qui lance tous les modèles**

In [22]:
def run_fit_models(X_train, y_train, X_test, y_test, columns):

    mse, rmse, r2, mae, elapsed = fit_dummyRegressor(X_train, y_train, X_test, y_test)
    scores_array = np.array([['DummyRegressor', mse, rmse, r2, mae, elapsed]])
    
    mse, rmse, r2, mae, elapsed = fit_ridge(X_train, y_train, X_test, y_test)
    scores_array = np.vstack([scores_array, ['ridge', mse, rmse, r2, mae, elapsed]])
    
    mse, rmse, r2, mae, elapsed = fit_lasso(X_train, y_train, X_test, y_test)
    scores_array = np.vstack([scores_array, ['lasso', mse, rmse, r2, mae, elapsed]])
    
    mse, rmse, r2, mae, elapsed = fit_elasticNet(X_train, y_train, X_test, y_test)
    scores_array = np.vstack([scores_array, ['elasticNet', mse, rmse, r2, mae, elapsed]])
    
    mse, rmse, r2, mae, elapsed = fit_ramdomForestRegressor(X_train, y_train, X_test, y_test, columns)
    scores_array = np.vstack([scores_array, ['RamdomForestRegressor', mse, rmse, r2, mae, elapsed]])
    
    mse, rmse, r2, mae, elapsed = fit_gradientBoostingRegressor(X_train, y_train, X_test, y_test)
    scores_array = np.vstack([scores_array, ['gradientBoostingRegressor', mse, rmse, r2, mae, elapsed]])
    
    mse, rmse, r2, mae, elapsed = fit_SVR(X_train, y_train, X_test, y_test)
    scores_array = np.vstack([scores_array, ['SVR', mse, rmse, r2, mae, elapsed]])

    return scores_array

### 1.2 - Fonction des modèles avec validation croisée

**Validation croisée avec le modèle Lasso**

In [25]:
def fit_GridSearchCV_lasso(X_train, y_train, scoring, param_grid):

    # Initialisation de GridSearchCV
    grid_search = GridSearchCV(
        estimator=Lasso(),           # une régression Lasso
        param_grid=param_grid,
        cv=5,                        # nombre de folds
        scoring=scoring,
        refit='R2',                  # Refit avec la meilleure valeur de R²
        #n_jobs=-1,                  # Utilisation de tous les cœurs disponibles
        verbose=1
    )

    # Entraînement de GridSearchCV
    grid_search.fit(X_train, y_train)

    return grid_search

**Validation croisée avec le modèle ElasticNet**

In [27]:
def fit_GridSearchCV_elasticNet(X_train, y_train, scoring, param_grid):

    # Initialisation de GridSearchCV
    grid_search = GridSearchCV(
        estimator=ElasticNet(),           # une régression ElasticNet
        param_grid=param_grid,
        cv=5,                        # nombre de folds
        scoring=scoring,
        refit='R2',                  # Refit avec la meilleure valeur de R²
        #n_jobs=-1,                  # Utilisation de tous les cœurs disponibles
        verbose=1
    )

    # Entraînement de GridSearchCV
    grid_search.fit(X_train, y_train)

    return grid_search

**Validation croisée avec le modèle GradientBoostingRegressor**

In [29]:
def fit_GridSearchCV_GradientBoostingRegressor(X_train, y_train, scoring, param_grid):

    # Configuration de GridSearchCV
    grid_search = GridSearchCV(
        estimator=GradientBoostingRegressor(random_state=42),
        param_grid=param_grid,
        scoring=scoring,
        cv=5,  # Validation croisée à 5 plis
        refit='R2',
        n_jobs=-1,  # Utilisation de tous les cœurs disponibles
        verbose=1  # Affichage des détails
    )

    # Entraînement de GridSearchCV
    grid_search.fit(X_train, y_train)

    return grid_search

**Validation croisée avec le modèle RamdomForestRegressor**

In [31]:
def fit_GridSearchCV_ramdomForestRegressor(X_train, y_train, scoring, param_grid):

    # Configuration de GridSearchCV
    grid_search = GridSearchCV(
        estimator=RandomForestRegressor(random_state=42),
        param_grid=param_grid,
        scoring=scoring,
        cv=5,  # Validation croisée à 5 plis
        refit='R2',
        n_jobs=-1,  # Utilisation de tous les cœurs disponibles
        verbose=1  # Affichage des détails
    )

    # Entraînement de GridSearchCV
    grid_search.fit(X_train, y_train)

    return grid_search

**Validation croisée avec SVR**

In [33]:
def fit_GridSearchCV_SVR(X_train, y_train, scoring, param_grid):

    # Configuration de GridSearchCV
    grid_search = GridSearchCV(
        estimator=SVR(),
        param_grid=param_grid,
        scoring=scoring,
        cv=5,  # Validation croisée à 5 plis
        refit='R2',
        n_jobs=-1,  # Utilisation de tous les cœurs disponibles
        verbose=1  # Affichage des détails
    )

    # Entraînement de GridSearchCV
    grid_search.fit(X_train, y_train)

    return grid_search

### 1.3 - Fonctions d'affichage des résultats

Affichage simple du résultat (meilleurs paramètres, scores,...) d'une validation croisée

In [35]:
def print_result_CV(grid_search):

    # Afficher les meilleurs paramètres trouvés
    print(f"Meilleurs paramètres : {grid_search.best_params_}")
    
    # Afficher le meilleur score
    print("Meilleu(s) score sur le jeu d'entraînement:")
    print(grid_search.best_score_)
    
    # Utiliser le modèle avec les meilleurs paramètres
    best_model = grid_search.best_estimator_
    
    # Afficher les performances correspondantes
    print("Résultats de la validation croisée :")
    for score_name in scoring.keys():
        
        print(f"\nScores pour '{score_name}':")    
        for mean, std, params, mean_fit_time in zip(
                grid_search.cv_results_[f'mean_test_{score_name}'],  # score moyen pour chaque score
                grid_search.cv_results_[f'std_test_{score_name}'],   # écart-type du score
                grid_search.cv_results_['params'],                   # valeur de l'hyperparamètre
                grid_search.cv_results_['mean_fit_time']             # temps moyen d'entraînement
        ):
            print(f"{score_name} = {mean:.3f} (+/-{std * 2:.03f}) for {params}")

Affichage sous forme de dataframe du résultat (meilleurs paramètres, scores,...) d'une validation croisée, pour plus de lisibilité

In [37]:
def print_result_CV_as_dataframe(grid_search, scoring):

    # Liste pour stocker les résultats
    results = []
    
    # Afficher les performances correspondantes
    print("Résultats de la validation croisée :")
    for score_name in scoring.keys():
           
        for mean, std, params, mean_fit_time in zip(
                grid_search.cv_results_[f'mean_test_{score_name}'],  # score moyen pour chaque score
                grid_search.cv_results_[f'std_test_{score_name}'],   # écart-type du score
                grid_search.cv_results_['params'],                   # valeur de l'hyperparamètre
                grid_search.cv_results_['mean_fit_time']             # temps moyen d'entraînement
        ):
                    
            # Ajouter chaque combinaison de résultats à une liste sous forme de dictionnaire
            results.append({
                "score_name": score_name,
                "mean_score": mean,
                "std_score": std,
                "params": params,
                "mean_fit_time": mean_fit_time
            })
    
    # Transformer en DataFrame
    df_results = pd.DataFrame(results)
    
    # Convertir la colonne 'params' en chaîne de caractères
    df_results['params'] = df_results['params'].apply(str)
    
    # Transformation avec pivot
    df_results = df_results.pivot(
        index='params',                             # Les paramètres deviennent l'index
        columns='score_name',                       # Les valeurs uniques de score_name deviennent des colonnes
        values=['mean_score', 'mean_fit_time']      # Les valeurs à remplir dans les colonnes (ici, mean_score)
    ).reset_index()
    
    # Aplatir les colonnes multi-indexées
    df_results.columns = ['_'.join(col).strip() for col in df_results.columns.values]
    
    # Réinitialiser l'index pour obtenir un DataFrame "normal"
    df_results = df_results.reset_index()
    df_results.drop(columns=['index'], inplace=True)
    
    # Supprimer l'axe des index
    df_results = df_results.rename_axis(None, axis=1)
    
    # On trie le dataframe sur la colonne R2 du pmus grand au plus petit
    df_results.sort_values(by='mean_score_R2', ascending=False, inplace=True)
    df_results = df_results.reset_index()
    df_results.drop(columns=['index', 'mean_fit_time_MAE', 'mean_fit_time_RMSE'], inplace=True)
    df_results.rename(columns={'mean_fit_time_R2': 'mean_fit_time'}, inplace=True)
    
    return df_results

Affichage des scores calculés sur le fichier de test avec le modèle de la meilleure performance de la validation croisée

In [39]:
def print_result_CV_on_test_file(X_test, y_test, grid_search):

    # Utiliser le modèle avec les meilleurs paramètres
    best_model = grid_search.best_estimator_
    
    # Prédictions avec le modèle optimisé
    y_pred = best_model.predict(X_test)
    
    # Évaluation du modèle avec différentes métriques
    mse = round(mean_squared_error(y_test, y_pred), 2)       # Erreur quadratique moyenne
    rmse = round(np.sqrt(mse), 2)                            # Racine carrée de l'erreur quadratique moyenne (RMSE)
    mae = round(mean_absolute_error(y_test, y_pred), 2)      # Erreur absolue moyenne
    r2 = round(r2_score(y_test, y_pred), 2)                  # Coefficient de détermination
    
    scores_cv_fe1 = np.array([[mse, rmse, r2, mae]])
    
    # Conversion de l'array en DataFrame
    df_scores_cv_fe1 = pd.DataFrame(scores_cv_fe1, columns=['MSE', 'RMSE', 'R2', 'MAE'])
    
    # on transforme la colonne R2 en numérique
    df_scores_cv_fe1['R2'] = pd.to_numeric(df_scores_cv_fe1['R2'], errors='coerce')
    
    # On trie le dataframe sur la colonne R2 du pmus grand au plus petit
    df_scores_cv_fe1.sort_values(by='R2', ascending=False, inplace=True)
    
    return df_scores_cv_fe1    

Affichage du tableau des scores des modèles sans validation croisée

In [41]:
def print_result_on_test_file(scores_array):

    # Conversion de l'array en DataFrame
    df = pd.DataFrame(scores_array, columns=['Modèle', 'MSE', 'RMSE', 'R2', 'MAE', 'ELAPSED_TIME'])
    
    # on transforme la colonne R2 en numérique
    df['R2'] = pd.to_numeric(df['R2'], errors='coerce')
    
    # On trie le dataframe sur la colonne R2 du pmus grand au plus petit
    df.sort_values(by='R2', ascending=False, inplace=True)
    df.reset_index(inplace=True)

    return df

Affichage des coeficients calculés dans un modèle Lasso

In [43]:
def print_coeffs_lasso(grid_search, columns):

    # Récupérer le meilleur modèle
    best_lasso = grid_search.best_estimator_
    
    # Extraire les coefficients
    coefficients = best_lasso.coef_
    
    # Associer les coefficients aux noms des variables
    coef_df = pd.DataFrame({
        'Feature': columns,
        'Coefficient': coefficients
    })
    
    coef_df.sort_values(by='Coefficient', ascending=False, inplace=True)
    
    # Afficher les coefficients
    print(coef_df)

### 1.4 - Paramètres

In [45]:
# Définition des hyperparamètres à tester
param_grid = {
    #'alpha': [0.01, 0.1, 1.0, 10.0],  # Différentes valeurs de régularisation
    'alpha': np.logspace(-6, 6, 13) 
}

In [46]:
pd.set_option('display.float_format', '{:.3f}'.format)  # Désactiver l'écriture scientifique

Créons une fonction pour calculer le RMSE qui n'a pas directement disponible dans le GridSearchCV :

In [48]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [49]:
rmse_scorer = make_scorer(rmse, greater_is_better=False)  # False car on minimise le RMSE

In [50]:
# Définition du dictionnaire des métriques de scoring
scoring = {
    'MAE': 'neg_mean_absolute_error',  # Utilise l'erreur absolue moyenne
    'R2': 'r2',                        # Utilise le coefficient de détermination
    'RMSE': rmse_scorer                # Utilise Root Mean Squared Error (racine carré de l'erreur quadratique moyenne)
}

## 2 - Développement et simulation du premier modèle (cible = SiteEnergyUseWN(kBtu))

In [52]:
# Charger le fichier de données
data_fe1 = pd.read_csv("C:/Users/admin/Documents/Projets/Projet_4/data_projet/cleaned/2016_Building_Energy_Benchmarking_fe1.csv", sep=',', low_memory=False)
data_fe1.head()

,NumberofBuildings,NumberofFloors,PropertyGFATotal,SiteEnergyUseWN(kBtu),TotalGHGEmissions,PrimaryPropertyType_Distribution Center,PrimaryPropertyType_Hospital,PrimaryPropertyType_Hotel,PrimaryPropertyType_K-12 School,PrimaryPropertyType_Laboratory,PrimaryPropertyType_Large Office,PrimaryPropertyType_Low-Rise Multifamily,PrimaryPropertyType_Medical Office,PrimaryPropertyType_Mixed Use Property,PrimaryPropertyType_Office,PrimaryPropertyType_Other,PrimaryPropertyType_Refrigerated Warehouse,PrimaryPropertyType_Residence Hall,PrimaryPropertyType_Restaurant,PrimaryPropertyType_Retail Store,PrimaryPropertyType_Self-Storage Facility,PrimaryPropertyType_Senior Care Community,PrimaryPropertyType_Small- and Mid-Sized Office,PrimaryPropertyType_Supermarket / Grocery Store,PrimaryPropertyType_University,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship Facility,Neighborhood_BALLARD,Neighborhood_CENTRAL,Neighborhood_DELRIDGE,Neighborhood_DELRIDGE NEIGHBORHOODS,Neighborhood_DOWNTOWN,Neighborhood_EAST,Neighborhood_GREATER DUWAMISH,Neighborhood_LAKE UNION,Neighborhood_MAGNOLIA / QUEEN ANNE,Neighborhood_NORTH,Neighborhood_NORTHEAST,Neighborhood_NORTHWEST,Neighborhood_SOUTHEAST,Neighborhood_SOUTHWEST,"YearBuilt_Bin_(1899.885, 1911.5]","YearBuilt_Bin_(1911.5, 1923.0]","YearBuilt_Bin_(1923.0, 1934.5]","YearBuilt_Bin_(1934.5, 1946.0]","YearBuilt_Bin_(1946.0, 1957.5]","YearBuilt_Bin_(1957.5, 1969.0]","YearBuilt_Bin_(1969.0, 1980.5]","YearBuilt_Bin_(1980.5, 1992.0]","YearBuilt_Bin_(1992.0, 2003.5]","YearBuilt_Bin_(2003.5, 2015.0]",electricity_percent,gaz_percent,steam_percent
0,1.000,12,88434.000,7456910.000,249.980,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,54.610,17.660,27.730
1,1.000,11,103566.000,8664479.000,295.860,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,38.660,61.340,0.000
2,1.000,10,61320.000,6946800.500,286.430,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,40.750,26.660,32.590
3,1.000,18,175580.000,14656503.000,505.010,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,37.880,62.120,0.000
4,1.000,2,97288.000,12581712.000,301.810,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,60.990,39.010,0.000


In [53]:
data_fe1.shape

(1444, 54)

In [54]:
data_fe1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1444 entries, 0 to 1443
Data columns (total 54 columns):
 #   Column                                           Non-Null Count  Dtype  
---  ------                                           --------------  -----  
 0   NumberofBuildings                                1444 non-null   float64
 1   NumberofFloors                                   1444 non-null   int64  
 2   PropertyGFATotal                                 1444 non-null   float64
 3   SiteEnergyUseWN(kBtu)                            1444 non-null   float64
 4   TotalGHGEmissions                                1444 non-null   float64
 5   PrimaryPropertyType_Distribution Center          1444 non-null   float64
 6   PrimaryPropertyType_Hospital                     1444 non-null   float64
 7   PrimaryPropertyType_Hotel                        1444 non-null   float64
 8   PrimaryPropertyType_K-12 School                  1444 non-null   float64
 9   PrimaryPropertyType_Laboratory

### 2.1 - Sélectionner les features et la cible :

In [56]:
y_fe1_conso = data_fe1['SiteEnergyUseWN(kBtu)']
X_fe1 = data_fe1.drop('SiteEnergyUseWN(kBtu)', axis=1, inplace=False)
X_fe1.shape

(1444, 53)

In [57]:
y_fe1_emissions = data_fe1['TotalGHGEmissions']
X_fe1 = X_fe1.drop('TotalGHGEmissions', axis=1, inplace=False)
X_fe1.shape

(1444, 52)

In [58]:
y_fe1_conso.shape

(1444,)

In [59]:
y_fe1_emissions.shape

(1444,)

### 2.2 - Standardiser les valeurs et créer les jeux d'entraînement / test

In [61]:
X_scale_fe1 = StandardScaler().fit_transform(X_fe1)

In [62]:
df = pd.DataFrame(X_scale_fe1)
df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51
count,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000,1444.000
mean,0.000,0.000,-0.000,-0.000,0.000,-0.000,-0.000,-0.000,0.000,0.000,0.000,0.000,-0.000,-0.000,-0.000,0.000,-0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000,-0.000,0.000,0.000,-0.000,0.000,-0.000,-0.000,0.000,-0.000,-0.000,-0.000,0.000,0.000,-0.000,0.000,0.000,0.000,-0.000
std,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
min,-0.133,-0.821,-0.838,-0.191,-0.026,-0.210,-0.305,-0.053,-0.277,-0.037,-0.141,-0.268,-0.046,-0.415,-0.088,-0.119,-0.092,-0.247,-0.141,-0.102,-0.495,-0.164,-0.102,-0.385,-0.221,-0.217,-0.183,-0.175,-0.026,-0.483,-0.277,-0.541,-0.304,-0.315,-0.215,-0.287,-0.239,-0.175,-0.164,-0.342,-0.291,-0.324,-0.185,-0.349,-0.433,-0.360,-0.347,-0.354,-0.312,-2.601,-1.092,-0.202
25%,-0.133,-0.554,-0.594,-0.191,-0.026,-0.210,-0.305,-0.053,-0.277,-0.037,-0.141,-0.268,-0.046,-0.415,-0.088,-0.119,-0.092,-0.247,-0.141,-0.102,-0.495,-0.164,-0.102,-0.385,-0.221,-0.217,-0.183,-0.175,-0.026,-0.483,-0.277,-0.541,-0.304,-0.315,-0.215,-0.287,-0.239,-0.175,-0.164,-0.342,-0.291,-0.324,-0.185,-0.349,-0.433,-0.360,-0.347,-0.354,-0.312,-0.804,-1.092,-0.202
50%,-0.133,-0.287,-0.367,-0.191,-0.026,-0.210,-0.305,-0.053,-0.277,-0.037,-0.141,-0.268,-0.046,-0.415,-0.088,-0.119,-0.092,-0.247,-0.141,-0.102,-0.495,-0.164,-0.102,-0.385,-0.221,-0.217,-0.183,-0.175,-0.026,-0.483,-0.277,-0.541,-0.304,-0.315,-0.215,-0.287,-0.239,-0.175,-0.164,-0.342,-0.291,-0.324,-0.185,-0.349,-0.433,-0.360,-0.347,-0.354,-0.312,0.001,-0.084,-0.202
75%,-0.133,0.247,0.138,-0.191,-0.026,-0.210,-0.305,-0.053,-0.277,-0.037,-0.141,-0.268,-0.046,-0.415,-0.088,-0.119,-0.092,-0.247,-0.141,-0.102,-0.495,-0.164,-0.102,-0.385,-0.221,-0.217,-0.183,-0.175,-0.026,-0.483,-0.277,-0.541,-0.304,-0.315,-0.215,-0.287,-0.239,-0.175,-0.164,-0.342,-0.291,-0.324,-0.185,-0.349,-0.433,-0.360,-0.347,-0.354,-0.312,1.159,0.818,-0.202
max,12.506,25.612,6.807,5.226,37.987,4.762,3.277,18.974,3.608,26.851,7.111,3.726,21.917,2.411,11.414,8.438,10.924,4.049,7.111,9.760,2.021,6.083,9.760,2.601,4.533,4.606,5.452,5.708,37.987,2.071,3.608,1.848,3.292,3.179,4.644,3.482,4.185,5.708,6.083,2.926,3.432,3.089,5.393,2.863,2.309,2.775,2.884,2.823,3.206,1.159,2.653,9.058


In [63]:
X_fe1_train, X_fe1_test, y_fe1_train, y_fe1_test = model_selection.train_test_split(X_scale_fe1, y_fe1_conso, test_size=0.25, random_state=42 ) # 25% des données dans le jeu de test

In [64]:
X_fe1_train.shape

(1083, 52)

In [65]:
X_fe1_test.shape

(361, 52)

In [66]:
y_fe1_train.shape

(1083,)

In [67]:
y_fe1_test.shape

(361,)

### 2.3 - Tests de modèles sans validation croisée

Il s'agit d'évaluer quelques modèles sans utiliser la validation croisée, en partant d'une baseline, pour aller vers des modèles plus élaborés.

Les hyperparamètres seront fixes ici. Ils seront automatisés par la suite par validation croisée avec GridSearchCV.

Les scores seront établis sur la base du fichier de tests. On fera une boucle sur chaque modèle, et on stockera les scrores dans un tableau. 

In [69]:
warnings.filterwarnings("ignore")

scores_array_fe1 = run_fit_models(X_fe1_train, y_fe1_train, X_fe1_test, y_fe1_test, X_fe1.columns)

Importance des features dans le RandomForestRegressor :
                                            Feature  Importance
2                                  PropertyGFATotal       0.505
21  PrimaryPropertyType_Supermarket / Grocery Store       0.088
49                              electricity_percent       0.062
1                                    NumberofFloors       0.049
50                                      gaz_percent       0.040
23                    PrimaryPropertyType_Warehouse       0.033
13                        PrimaryPropertyType_Other       0.025
7                    PrimaryPropertyType_Laboratory       0.018
16                   PrimaryPropertyType_Restaurant       0.013
19        PrimaryPropertyType_Senior Care Community       0.012
47                   YearBuilt_Bin_(1992.0, 2003.5]       0.010
45                   YearBuilt_Bin_(1969.0, 1980.5]       0.010
29                            Neighborhood_DOWNTOWN       0.009
11           PrimaryPropertyType_Mixed Use Prope

Affichons le résultat dans un dataframe, triès par ordre décroissant sur R2 :

Plus R2 est proche de 1, plus le modèle est performant.

In [71]:
df_results_fe1 = print_result_on_test_file(scores_array_fe1)
df_results_fe1.head(10)

,index,Modèle,MSE,RMSE,R2,MAE,ELAPSED_TIME
0,1,ridge,4883143028088.79,2209783.48,0.650,1526074.66,0.022
1,2,lasso,4882121725575.46,2209552.38,0.650,1525093.94,0.043
2,4,RamdomForestRegressor,5056831827541.21,2248740.05,0.640,1489276.19,1.071
3,5,gradientBoostingRegressor,5513470357850.86,2348078.01,0.610,1532356.7,0.245
4,3,elasticNet,5751242024803.79,2398174.73,0.590,1751934.71,0.002
5,0,DummyRegressor,14143195564884.4,3760744.02,-0.010,2849262.3,0.001
6,6,SVR,16828270384158.12,4102227.49,-0.200,2693551.59,0.118


Sans validation croisée et optimisation des hyperparamètres, les modèles Ridge et Lasso sont les plus performants (R2 = 0.65), suivi de près par le RandomForestRegressor (R2 = 0.64).

### 2.4 - Validation croisée avec le modèle Lasso

In [74]:
# Définition des hyperparamètres à tester
param_grid = {
    'alpha': np.logspace(-6, 6, 13) 
}

In [75]:
grid_search_fe1 = fit_GridSearchCV_lasso(X_fe1_train, y_fe1_train, scoring, param_grid)

Fitting 5 folds for each of 13 candidates, totalling 65 fits


In [76]:
print_result_CV(grid_search_fe1)

Meilleurs paramètres : {'alpha': 10000.0}
Meilleu(s) score sur le jeu d'entraînement:
0.5904473335057308
Résultats de la validation croisée :

Scores pour 'MAE':
MAE = -1542554.704 (+/-189210.398) for {'alpha': 1e-06}
MAE = -1542554.704 (+/-189210.398) for {'alpha': 1e-05}
MAE = -1542554.704 (+/-189210.399) for {'alpha': 0.0001}
MAE = -1542554.705 (+/-189210.403) for {'alpha': 0.001}
MAE = -1542554.709 (+/-189210.443) for {'alpha': 0.01}
MAE = -1542554.755 (+/-189210.850) for {'alpha': 0.1}
MAE = -1542555.208 (+/-189214.913) for {'alpha': 1.0}
MAE = -1542560.476 (+/-189256.227) for {'alpha': 10.0}
MAE = -1542596.487 (+/-189688.543) for {'alpha': 100.0}
MAE = -1542196.483 (+/-190822.699) for {'alpha': 1000.0}
MAE = -1535455.447 (+/-195365.197) for {'alpha': 10000.0}
MAE = -1536052.141 (+/-216132.916) for {'alpha': 100000.0}
MAE = -2176110.008 (+/-279706.375) for {'alpha': 1000000.0}

Scores pour 'R2':
R2 = 0.589 (+/-0.115) for {'alpha': 1e-06}
R2 = 0.589 (+/-0.115) for {'alpha': 1e-05}


In [77]:
print_result_CV_as_dataframe(grid_search_fe1, scoring).head(30)

Résultats de la validation croisée :


,params_,mean_score_MAE,mean_score_R2,mean_score_RMSE,mean_fit_time
0,{'alpha': 10000.0},-1535455.447,0.590,-2252862.327,0.003
1,{'alpha': 1000.0},-1542196.483,0.589,-2256989.725,0.009
2,{'alpha': 1e-06},-1542554.704,0.589,-2257262.565,0.035
3,{'alpha': 1e-05},-1542554.704,0.589,-2257262.565,0.034
4,{'alpha': 0.0001},-1542554.704,0.589,-2257262.565,0.037
5,{'alpha': 0.001},-1542554.705,0.589,-2257262.565,0.031
6,{'alpha': 0.01},-1542554.709,0.589,-2257262.566,0.030
7,{'alpha': 0.1},-1542554.755,0.589,-2257262.571,0.031
8,{'alpha': 1.0},-1542555.208,0.589,-2257262.619,0.025
9,{'alpha': 10.0},-1542560.476,0.589,-2257262.656,0.032


Le modèle Lasso est le plus performant avec alpha = 10 000 (R2 = 0.590). C'est aussi pour cette valeur qu'il est le plus rapide.

Affichons le résultat sur le fichier de test :

In [79]:
print_result_CV_on_test_file(X_fe1_test, y_fe1_test, grid_search_fe1).head(30)

,MSE,RMSE,R2,MAE
0,4856026422771.060,2203639.360,0.650,1517734.680


Le coefficient de détermination R2 est meilleur sur le fichier de test (0.65).

Regardons l'importance des coeeficients de chaque feature :

In [82]:
print_coeffs_lasso(grid_search_fe1, X_fe1.columns)

                                            Feature  Coefficient
2                                  PropertyGFATotal  2227812.959
21  PrimaryPropertyType_Supermarket / Grocery Store  1034630.413
7                    PrimaryPropertyType_Laboratory   389883.927
13                        PrimaryPropertyType_Other   301894.227
16                   PrimaryPropertyType_Restaurant   236024.477
29                            Neighborhood_DOWNTOWN   196833.533
19        PrimaryPropertyType_Senior Care Community   191051.158
5                         PrimaryPropertyType_Hotel   188542.058
47                   YearBuilt_Bin_(1992.0, 2003.5]   187773.024
45                   YearBuilt_Bin_(1969.0, 1980.5]   162743.352
0                                 NumberofBuildings   150340.353
10               PrimaryPropertyType_Medical Office   127639.771
8                  PrimaryPropertyType_Large Office   125934.169
1                                    NumberofFloors   125089.962
51                       

Les 2 plus fortes contribution positive sont nettement la surface totale, et le PrimaryPropertyType_Supermarket / Grocery Store.

Dans l'étape suivante, on va essayer d'améliorer le score R2 des validations croisées.

## 3 - Amélioration du feature Engineering (cible = SiteEnergyUseWN(kBtu))

### 3.1 - 2ème feature Engineering

In [86]:
# Charger le fichier de données
data_fe2 = pd.read_csv("C:/Users/admin/Documents/Projets/Projet_4/data_projet/cleaned/2016_Building_Energy_Benchmarking_fe2.csv", sep=',', low_memory=False)
data_fe2.head()

,NumberofBuildings,NumberofFloors,PropertyGFAParking,SiteEnergyUseWN(kBtu),TotalGHGEmissions,Neighborhood_BALLARD,Neighborhood_CENTRAL,Neighborhood_DELRIDGE,Neighborhood_DELRIDGE NEIGHBORHOODS,Neighborhood_DOWNTOWN,Neighborhood_EAST,Neighborhood_GREATER DUWAMISH,Neighborhood_LAKE UNION,Neighborhood_MAGNOLIA / QUEEN ANNE,Neighborhood_NORTH,Neighborhood_NORTHEAST,Neighborhood_NORTHWEST,Neighborhood_SOUTHEAST,Neighborhood_SOUTHWEST,"YearBuilt_Bin_(1899.885, 1911.5]","YearBuilt_Bin_(1911.5, 1923.0]","YearBuilt_Bin_(1923.0, 1934.5]","YearBuilt_Bin_(1934.5, 1946.0]","YearBuilt_Bin_(1946.0, 1957.5]","YearBuilt_Bin_(1957.5, 1969.0]","YearBuilt_Bin_(1969.0, 1980.5]","YearBuilt_Bin_(1980.5, 1992.0]","YearBuilt_Bin_(1992.0, 2003.5]","YearBuilt_Bin_(2003.5, 2015.0]",PrimaryPropertyType_Distribution Center,PrimaryPropertyType_Hospital,PrimaryPropertyType_Hotel,PrimaryPropertyType_K-12 School,PrimaryPropertyType_Laboratory,PrimaryPropertyType_Large Office,PrimaryPropertyType_Low-Rise Multifamily,PrimaryPropertyType_Medical Office,PrimaryPropertyType_Mixed Use Property,PrimaryPropertyType_Office,PrimaryPropertyType_Other,PrimaryPropertyType_Refrigerated Warehouse,PrimaryPropertyType_Residence Hall,PrimaryPropertyType_Restaurant,PrimaryPropertyType_Retail Store,PrimaryPropertyType_Self-Storage Facility,PrimaryPropertyType_Senior Care Community,PrimaryPropertyType_Small- and Mid-Sized Office,PrimaryPropertyType_Supermarket / Grocery Store,PrimaryPropertyType_University,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship Facility,electricity_percent,gaz_percent,steam_percent,usage_Autres,usage_Bureaux & Espaces de travail,usage_Commerce & Retail,usage_Entrepôts et Logistique,usage_Hébergement & Logement,usage_Loisirs et Divertissement,usage_Restauration,usage_Services publics & Infrastructure,usage_Soins médicaux,usage_Transports & Parking,usage_Éducation,PropertyGFAOutsideParking
0,1.000,12,0,7456910.000,249.980,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,54.610,17.660,27.730,0.000,0.000,0.000,0.000,100.000,0.000,0.000,0.000,0.000,0.000,0.000,88434.000
1,1.000,11,15064,8664479.000,295.860,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,38.660,61.340,0.000,0.000,0.000,0.000,0.000,80.990,0.000,4.460,0.000,0.000,14.550,0.000,88502.000
2,1.000,10,0,6946800.500,286.430,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,40.750,26.660,32.590,0.000,0.000,0.000,0.000,100.000,0.000,0.000,0.000,0.000,0.000,0.000,61320.000
3,1.000,18,62000,14656503.000,505.010,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,37.880,62.120,0.000,0.000,0.000,0.000,0.000,64.480,0.000,0.000,0.000,0.000,35.520,0.000,113580.000
4,1.000,2,37198,12581712.000,301.810,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,60.990,39.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,100.000,0.000,0.000,0.000,60090.000


In [87]:
data_fe2.shape

(1440, 66)

**Sélectionner les features et la cible :**

In [89]:
y_fe2_conso = data_fe2['SiteEnergyUseWN(kBtu)']
X_fe2 = data_fe2.drop('SiteEnergyUseWN(kBtu)', axis=1, inplace=False)
X_fe2.shape

(1440, 65)

In [90]:
y_fe2_emissions = data_fe2['TotalGHGEmissions']
X_fe2 = X_fe2.drop('TotalGHGEmissions', axis=1, inplace=False)
X_fe2.shape

(1440, 64)

**Standardiser les valeurs et créer les jeux d'entraînement / test**

In [92]:
X_scale_fe2 = StandardScaler().fit_transform(X_fe2)

df = pd.DataFrame(X_scale_fe2)
df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63
count,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000
mean,0.000,0.000,0.000,0.000,-0.000,-0.000,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,-0.000,-0.000,0.000,0.000,-0.000,-0.000,-0.000,-0.000,0.000,-0.000,-0.000,-0.000,0.000,0.000,0.000,0.000,-0.000,-0.000,0.000,0.000,-0.000,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,0.000,-0.000,-0.000,0.000,-0.000,0.000,0.000,-0.000,0.000,-0.000,0.000,-0.000
std,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
min,-0.132,-0.820,-0.288,-0.216,-0.184,-0.175,-0.026,-0.481,-0.278,-0.542,-0.303,-0.315,-0.216,-0.288,-0.239,-0.175,-0.165,-0.341,-0.290,-0.324,-0.184,-0.350,-0.434,-0.361,-0.347,-0.355,-0.311,-0.192,-0.026,-0.209,-0.306,-0.053,-0.278,-0.037,-0.141,-0.269,-0.046,-0.415,-0.088,-0.119,-0.088,-0.247,-0.138,-0.103,-0.495,-0.165,-0.103,-0.385,-0.221,-2.599,-1.092,-0.202,-0.269,-0.761,-0.414,-0.523,-0.301,-0.159,-0.158,-0.296,-0.094,-0.367,-0.345,-1.049
25%,-0.132,-0.553,-0.288,-0.216,-0.184,-0.175,-0.026,-0.481,-0.278,-0.542,-0.303,-0.315,-0.216,-0.288,-0.239,-0.175,-0.165,-0.341,-0.290,-0.324,-0.184,-0.350,-0.434,-0.361,-0.347,-0.355,-0.311,-0.192,-0.026,-0.209,-0.306,-0.053,-0.278,-0.037,-0.141,-0.269,-0.046,-0.415,-0.088,-0.119,-0.088,-0.247,-0.138,-0.103,-0.495,-0.165,-0.103,-0.385,-0.221,-0.803,-1.092,-0.202,-0.269,-0.761,-0.414,-0.523,-0.301,-0.159,-0.158,-0.296,-0.094,-0.367,-0.345,-0.622
50%,-0.132,-0.286,-0.288,-0.216,-0.184,-0.175,-0.026,-0.481,-0.278,-0.542,-0.303,-0.315,-0.216,-0.288,-0.239,-0.175,-0.165,-0.341,-0.290,-0.324,-0.184,-0.350,-0.434,-0.361,-0.347,-0.355,-0.311,-0.192,-0.026,-0.209,-0.306,-0.053,-0.278,-0.037,-0.141,-0.269,-0.046,-0.415,-0.088,-0.119,-0.088,-0.247,-0.138,-0.103,-0.495,-0.165,-0.103,-0.385,-0.221,0.001,-0.084,-0.202,-0.269,-0.761,-0.414,-0.523,-0.301,-0.159,-0.158,-0.296,-0.094,-0.367,-0.345,-0.357
75%,-0.132,0.247,-0.288,-0.216,-0.184,-0.175,-0.026,-0.481,-0.278,-0.542,-0.303,-0.315,-0.216,-0.288,-0.239,-0.175,-0.165,-0.341,-0.290,-0.324,-0.184,-0.350,-0.434,-0.361,-0.347,-0.355,-0.311,-0.192,-0.026,-0.209,-0.306,-0.053,-0.278,-0.037,-0.141,-0.269,-0.046,-0.415,-0.088,-0.119,-0.088,-0.247,-0.138,-0.103,-0.495,-0.165,-0.103,-0.385,-0.221,1.159,0.818,-0.202,-0.269,0.913,-0.414,-0.523,-0.301,-0.159,-0.158,-0.296,-0.094,-0.367,-0.345,0.178
max,12.515,25.594,12.040,4.637,5.444,5.700,37.934,2.077,3.603,1.845,3.302,3.174,4.637,3.477,4.179,5.700,6.074,2.933,3.443,3.084,5.444,2.859,2.306,2.770,2.879,2.819,3.215,5.219,37.934,4.796,3.272,18.947,3.603,26.814,7.101,3.721,21.886,2.407,11.398,8.426,11.398,4.043,7.234,9.747,2.022,6.074,9.747,2.597,4.527,1.159,2.652,9.046,4.509,1.834,3.037,2.199,3.859,8.574,10.992,3.805,11.920,6.989,2.988,5.912


In [93]:
# 25% des données dans le jeu de test
X_fe2_train, X_fe2_test, y_fe2_train, y_fe2_test = model_selection.train_test_split(X_scale_fe2, y_fe2_conso, test_size=0.25, random_state=42 )

**Tests de modèles sans validation croisée**

In [95]:
warnings.filterwarnings("ignore")

scores_array_fe2 = run_fit_models(X_fe2_train, y_fe2_train, X_fe2_test, y_fe2_test, X_fe2.columns)

Importance des features dans le RandomForestRegressor :
                                            Feature  Importance
63                        PropertyGFAOutsideParking       0.461
45  PrimaryPropertyType_Supermarket / Grocery Store       0.079
55                    usage_Entrepôts et Logistique       0.078
49                              electricity_percent       0.036
1                                    NumberofFloors       0.032
50                                      gaz_percent       0.028
2                                PropertyGFAParking       0.027
58                               usage_Restauration       0.027
60                             usage_Soins médicaux       0.024
53               usage_Bureaux & Espaces de travail       0.015
54                          usage_Commerce & Retail       0.013
57                  usage_Loisirs et Divertissement       0.013
52                                     usage_Autres       0.011
61                       usage_Transports & Park

In [96]:
df_results_fe2 = print_result_on_test_file(scores_array_fe2)
df_results_fe2.head(10)

,index,Modèle,MSE,RMSE,R2,MAE,ELAPSED_TIME
0,5,gradientBoostingRegressor,4480118582382.66,2116629.06,0.690,1416498.5,0.353
1,4,RamdomForestRegressor,4759592913217.16,2181649.13,0.670,1453670.51,1.216
2,1,ridge,5095680824169.57,2257361.47,0.650,1558394.94,0.003
3,2,lasso,5094274788898.77,2257050.02,0.650,1558245.43,0.039
4,3,elasticNet,5654405660714.58,2377899.42,0.610,1691624.38,0.002
5,0,DummyRegressor,14578031018368.1,3818118.78,-0.010,2862093.8,0.001
6,6,SVR,17300527736892.97,4159390.31,-0.190,2674589.47,0.13


On observe que les modèles Ridge et Lasso sont moins performants que les modèles gradientBoostingRegressor et RamdomForestRegressor. En fait c'est surtout la performance de Ridge et Lasso qui a chuté avec le 2ème feature engineering par rapport au 1er.

**Validation croisée avec le modèle Lasso**

In [99]:
# Définition des hyperparamètres à tester
param_grid = {
    'alpha': np.logspace(-7, 7, 13) 
}

In [100]:
grid_search_fe2 = fit_GridSearchCV_lasso(X_fe2_train, y_fe2_train, scoring, param_grid)
print_result_CV(grid_search_fe2)

Fitting 5 folds for each of 13 candidates, totalling 65 fits
Meilleurs paramètres : {'alpha': 46415.888336127915}
Meilleu(s) score sur le jeu d'entraînement:
0.61707799982737
Résultats de la validation croisée :

Scores pour 'MAE':
MAE = -1478861.576 (+/-195402.552) for {'alpha': 1e-07}
MAE = -1478861.576 (+/-195402.552) for {'alpha': 1.4677992676220705e-06}
MAE = -1478861.576 (+/-195402.552) for {'alpha': 2.1544346900318867e-05}
MAE = -1478861.576 (+/-195402.552) for {'alpha': 0.00031622776601683794}
MAE = -1478861.578 (+/-195402.556) for {'alpha': 0.004641588833612782}
MAE = -1478861.613 (+/-195402.620) for {'alpha': 0.06812920690579623}
MAE = -1478862.143 (+/-195403.586) for {'alpha': 1.0}
MAE = -1478863.922 (+/-195417.232) for {'alpha': 14.677992676220736}
MAE = -1478744.989 (+/-195651.470) for {'alpha': 215.44346900318865}
MAE = -1476371.658 (+/-198291.535) for {'alpha': 3162.2776601683795}
MAE = -1462772.789 (+/-176282.099) for {'alpha': 46415.888336127915}
MAE = -1980849.083 (+/

In [101]:
print_result_CV_as_dataframe(grid_search_fe2, scoring).head(30)

Résultats de la validation croisée :


,params_,mean_score_MAE,mean_score_R2,mean_score_RMSE,mean_fit_time
0,{'alpha': 46415.888336127915},-1462772.789,0.617,-2159872.977,0.003
1,{'alpha': 3162.2776601683795},-1476371.658,0.614,-2166861.183,0.005
2,{'alpha': 215.44346900318865},-1478744.989,0.613,-2169365.900,0.038
3,{'alpha': 14.677992676220736},-1478863.922,0.613,-2169564.985,0.039
4,{'alpha': 1.0},-1478862.143,0.613,-2169578.331,0.045
5,{'alpha': 0.06812920690579623},-1478861.613,0.613,-2169578.964,0.040
6,{'alpha': 0.004641588833612782},-1478861.578,0.613,-2169579.009,0.047
7,{'alpha': 0.00031622776601683794},-1478861.576,0.613,-2169579.012,0.031
8,{'alpha': 2.1544346900318867e-05},-1478861.576,0.613,-2169579.012,0.035
9,{'alpha': 1.4677992676220705e-06},-1478861.576,0.613,-2169579.012,0.036


Le meilleur score R2 de la validation croisée est un peu mieux que dans le 1er feature Engineering.

Et avec le fichier de test :

In [103]:
print_result_CV_on_test_file(X_fe2_test, y_fe2_test, grid_search_fe2).head(30)

,MSE,RMSE,R2,MAE
0,4822406331270.100,2195997.800,0.670,1488521.690


In [104]:
print_coeffs_lasso(grid_search_fe2, X_fe2.columns)

                                            Feature  Coefficient
63                        PropertyGFAOutsideParking  1988305.992
45  PrimaryPropertyType_Supermarket / Grocery Store   944927.231
2                                PropertyGFAParking   503435.560
31                   PrimaryPropertyType_Laboratory   274986.684
58                               usage_Restauration   268569.967
25                   YearBuilt_Bin_(1992.0, 2003.5]   238768.928
43        PrimaryPropertyType_Senior Care Community   213816.749
60                             usage_Soins médicaux   192325.699
57                  usage_Loisirs et Divertissement   187922.172
37                        PrimaryPropertyType_Other   162482.645
23                   YearBuilt_Bin_(1969.0, 1980.5]   154950.289
34               PrimaryPropertyType_Medical Office   154179.307
51                                    steam_percent   144486.179
38       PrimaryPropertyType_Refrigerated Warehouse   137682.396
29                       

Regardons si une validation croisée avec le modèle GradientBoostingRegressor ferait mieux.

**Validation croisée avec le modèle GradientBoostingRegressor**

In [106]:
# Définition des hyperparamètres pour la recherche
param_grid = {
    'n_estimators': [50, 100, 150],  # Nombre d'arbres dans l'ensemble.
    'learning_rate': [0.01, 0.1, 0.2], # Taux d'apprentissage pour la réduction du poids de chaque arbre
    'max_depth': [3, 5, 7], # Profondeur maximale de chaque arbre
    'subsample': [0.8, 1.0] # Fraction des échantillons utilisés pour entraîner chaque arbre.
}

In [107]:
grid_search_fe2_gradient = fit_GridSearchCV_GradientBoostingRegressor(X_fe2_train, y_fe2_train, scoring, param_grid)
print_result_CV(grid_search_fe2_gradient)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Meilleurs paramètres : {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Meilleu(s) score sur le jeu d'entraînement:
0.6342091757531982
Résultats de la validation croisée :

Scores pour 'MAE':
MAE = -2173277.019 (+/-261855.158) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}
MAE = -2177525.091 (+/-270005.484) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50, 'subsample': 1.0}
MAE = -1897385.953 (+/-213993.133) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
MAE = -1908475.468 (+/-224864.822) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
MAE = -1748779.611 (+/-181119.918) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 150, 'subsample': 0.8}
MAE = -1768781.832 (+/-182641.435) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 150, 'subsample': 1.0}
MAE = -2112

Affichons le résultat de la validation croisée :

In [109]:
print_result_CV_as_dataframe(grid_search_fe2_gradient, scoring).head(30)

Résultats de la validation croisée :


,params_,mean_score_MAE,mean_score_R2,mean_score_RMSE,mean_fit_time
0,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",-1389995.643,0.634,-2106896.086,0.644
1,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",-1391892.298,0.632,-2113880.335,0.906
2,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",-1405946.893,0.632,-2115455.635,0.738
3,"{'learning_rate': 0.2, 'max_depth': 3, 'n_esti...",-1406688.937,0.631,-2116205.139,0.280
4,"{'learning_rate': 0.2, 'max_depth': 3, 'n_esti...",-1401008.154,0.631,-2118600.975,0.346
5,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",-1382381.030,0.631,-2120243.692,1.037
6,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",-1395471.969,0.630,-2121878.966,0.578
7,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",-1403296.954,0.630,-2120790.799,1.291
8,"{'learning_rate': 0.2, 'max_depth': 3, 'n_esti...",-1411471.738,0.627,-2129571.658,0.665
9,"{'learning_rate': 0.2, 'max_depth': 3, 'n_esti...",-1414874.430,0.624,-2133977.233,0.568


Et avec le fichier de test :

In [111]:
print_result_CV_on_test_file(X_fe2_test, y_fe2_test, grid_search_fe2_gradient).head(30)

,MSE,RMSE,R2,MAE
0,4605739041357.600,2146098.560,0.680,1418104.970


La validation croisée avec le modèle GradientBoostingRegressor est bien meilleure.

### 3.2 - 3ème feature Engineering

On utilise les mêmes features que dans le 2ème feature engineering, mais on a utilisé le TargetEncoder pour numériser les variables catégorielles, en choisissant la variable SiteEnergyUseWN(kBtu) comme cible (target).

In [114]:
# Charger le fichier de données
data_fe3 = pd.read_csv("C:/Users/admin/Documents/Projets/Projet_4/data_projet/cleaned/2016_Building_Energy_Benchmarking_fe3.csv", sep=',', low_memory=False)
data_fe3.head()

,NumberofBuildings,NumberofFloors,PropertyGFAParking,SiteEnergyUseWN(kBtu),TotalGHGEmissions,Neighborhood,YearBuilt_Bin,PrimaryPropertyType,electricity_percent,gaz_percent,steam_percent,usage_Autres,usage_Bureaux & Espaces de travail,usage_Commerce & Retail,usage_Entrepôts et Logistique,usage_Hébergement & Logement,usage_Loisirs et Divertissement,usage_Restauration,usage_Services publics & Infrastructure,usage_Soins médicaux,usage_Transports & Parking,usage_Éducation,PropertyGFAOutsideParking
0,1.000,12,0,7456910.000,249.980,5000274.943,3533085.625,6218125.878,54.610,17.660,27.730,0.000,0.000,0.000,0.000,100.000,0.000,0.000,0.000,0.000,0.000,0.000,88434.000
1,1.000,11,15064,8664479.000,295.860,5000274.943,6145114.477,6218125.878,38.660,61.340,0.000,0.000,0.000,0.000,0.000,80.990,0.000,4.460,0.000,0.000,14.550,0.000,88502.000
2,1.000,10,0,6946800.500,286.430,5000274.943,3533085.625,6218125.878,40.750,26.660,32.590,0.000,0.000,0.000,0.000,100.000,0.000,0.000,0.000,0.000,0.000,0.000,61320.000
3,1.000,18,62000,14656503.000,505.010,5000274.943,3595235.070,6218125.878,37.880,62.120,0.000,0.000,0.000,0.000,0.000,64.480,0.000,0.000,0.000,0.000,35.520,0.000,113580.000
4,1.000,2,37198,12581712.000,301.810,5000274.943,6145114.477,4209164.996,60.990,39.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,100.000,0.000,0.000,0.000,60090.000


In [115]:
data_fe3.shape

(1440, 23)

**Sélectionner les features et la cible :**

In [117]:
y_fe3_conso = data_fe3['SiteEnergyUseWN(kBtu)']
X_fe3 = data_fe3.drop('SiteEnergyUseWN(kBtu)', axis=1, inplace=False)
X_fe3.shape

(1440, 22)

In [118]:
y_fe3_emissions = data_fe3['TotalGHGEmissions']
X_fe3 = X_fe3.drop('TotalGHGEmissions', axis=1, inplace=False)
X_fe3.shape

(1440, 21)

**Standardiser les valeurs et créer les jeux d'entraînement / test**

In [120]:
X_scale_fe3 = StandardScaler().fit_transform(X_fe3)

df = pd.DataFrame(X_scale_fe3)
df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
count,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000
mean,0.000,0.000,0.000,0.000,-0.000,-0.000,0.000,0.000,-0.000,0.000,-0.000,-0.000,0.000,-0.000,0.000,0.000,-0.000,0.000,-0.000,0.000,-0.000
std,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
min,-0.132,-0.820,-0.288,-1.394,-1.193,-1.150,-2.599,-1.092,-0.202,-0.269,-0.761,-0.414,-0.523,-0.301,-0.159,-0.158,-0.296,-0.094,-0.367,-0.345,-1.049
25%,-0.132,-0.553,-0.288,-0.851,-0.496,-0.562,-0.803,-1.092,-0.202,-0.269,-0.761,-0.414,-0.523,-0.301,-0.159,-0.158,-0.296,-0.094,-0.367,-0.345,-0.622
50%,-0.132,-0.286,-0.288,-0.094,-0.219,-0.134,0.001,-0.084,-0.202,-0.269,-0.761,-0.414,-0.523,-0.301,-0.159,-0.158,-0.296,-0.094,-0.367,-0.345,-0.357
75%,-0.132,0.247,-0.288,0.953,0.156,0.237,1.159,0.818,-0.202,-0.269,0.913,-0.414,-0.523,-0.301,-0.159,-0.158,-0.296,-0.094,-0.367,-0.345,0.178
max,12.515,25.594,12.040,1.511,2.165,2.703,1.159,2.652,9.046,4.509,1.834,3.037,2.199,3.859,8.574,10.992,3.805,11.920,6.989,2.988,5.912


In [121]:
# 25% des données dans le jeu de test
X_fe3_train, X_fe3_test, y_fe3_train, y_fe3_test = model_selection.train_test_split(X_scale_fe3, y_fe3_conso, test_size=0.25, random_state=42 )

**Tests de modèles sans validation croisée**

In [123]:
warnings.filterwarnings("ignore")

scores_array_fe3 = run_fit_models(X_fe3_train, y_fe3_train, X_fe3_test, y_fe3_test, X_fe3.columns)

Importance des features dans le RandomForestRegressor :
                                    Feature  Importance
20                PropertyGFAOutsideParking       0.420
5                       PrimaryPropertyType       0.223
6                       electricity_percent       0.039
12            usage_Entrepôts et Logistique       0.038
1                            NumberofFloors       0.032
4                             YearBuilt_Bin       0.032
7                               gaz_percent       0.032
15                       usage_Restauration       0.027
3                              Neighborhood       0.021
10       usage_Bureaux & Espaces de travail       0.021
2                        PropertyGFAParking       0.019
17                     usage_Soins médicaux       0.017
18               usage_Transports & Parking       0.016
11                  usage_Commerce & Retail       0.015
9                              usage_Autres       0.010
13             usage_Hébergement & Logement     

In [124]:
df_results_fe3 = print_result_on_test_file(scores_array_fe3)
df_results_fe3.head(10)

,index,Modèle,MSE,RMSE,R2,MAE,ELAPSED_TIME
0,4,RamdomForestRegressor,4825013493694.9,2196591.34,0.670,1479713.41,0.878
1,5,gradientBoostingRegressor,4725100210860.37,2173729.56,0.670,1461033.21,0.266
2,1,ridge,5437655932273.71,2331878.2,0.620,1621716.58,0.001
3,2,lasso,5438454742533.71,2332049.47,0.620,1621783.9,0.017
4,3,elasticNet,5963744420152.16,2442077.89,0.590,1744748.96,0.001
5,0,DummyRegressor,14578031018368.1,3818118.78,-0.010,2862093.8,0.0
6,6,SVR,17300285747579.89,4159361.22,-0.190,2674575.44,0.106


C'est encore le GradientBoostingRegressor qui est le meilleur sans la validation croisée.

**Validation croisée avec le modèle Lasso**

In [127]:
# Définition des hyperparamètres à tester
param_grid = {
    'alpha': np.logspace(-7, 7, 13) 
}

grid_search_fe3 = fit_GridSearchCV_lasso(X_fe3_train, y_fe3_train, scoring, param_grid)
print_result_CV(grid_search_fe3)

Fitting 5 folds for each of 13 candidates, totalling 65 fits
Meilleurs paramètres : {'alpha': 3162.2776601683795}
Meilleu(s) score sur le jeu d'entraînement:
0.5823467408784783
Résultats de la validation croisée :

Scores pour 'MAE':
MAE = -1548499.893 (+/-187466.844) for {'alpha': 1e-07}
MAE = -1548499.893 (+/-187466.844) for {'alpha': 1.4677992676220705e-06}
MAE = -1548499.893 (+/-187466.844) for {'alpha': 2.1544346900318867e-05}
MAE = -1548499.893 (+/-187466.844) for {'alpha': 0.00031622776601683794}
MAE = -1548499.893 (+/-187466.845) for {'alpha': 0.004641588833612782}
MAE = -1548499.891 (+/-187466.857) for {'alpha': 0.06812920690579623}
MAE = -1548499.871 (+/-187467.037) for {'alpha': 1.0}
MAE = -1548499.460 (+/-187469.346) for {'alpha': 14.677992676220736}
MAE = -1548490.162 (+/-187518.561) for {'alpha': 215.44346900318865}
MAE = -1548460.518 (+/-188245.953) for {'alpha': 3162.2776601683795}
MAE = -1552097.397 (+/-194878.923) for {'alpha': 46415.888336127915}
MAE = -1874795.798 (

In [128]:
print_result_CV_as_dataframe(grid_search_fe3, scoring).head(30)

Résultats de la validation croisée :


,params_,mean_score_MAE,mean_score_R2,mean_score_RMSE,mean_fit_time
0,{'alpha': 3162.2776601683795},-1548460.518,0.582,-2258731.892,0.000
1,{'alpha': 215.44346900318865},-1548490.162,0.582,-2259210.089,0.006
2,{'alpha': 14.677992676220736},-1548499.460,0.582,-2259251.594,0.009
3,{'alpha': 1.0},-1548499.871,0.582,-2259254.115,0.012
4,{'alpha': 0.06812920690579623},-1548499.891,0.582,-2259254.276,0.009
5,{'alpha': 0.004641588833612782},-1548499.893,0.582,-2259254.287,0.009
6,{'alpha': 0.00031622776601683794},-1548499.893,0.582,-2259254.287,0.009
7,{'alpha': 2.1544346900318867e-05},-1548499.893,0.582,-2259254.288,0.013
8,{'alpha': 1.4677992676220705e-06},-1548499.893,0.582,-2259254.288,0.010
9,{'alpha': 1e-07},-1548499.893,0.582,-2259254.288,0.010


Les résultats avec le modèle Lasso sont moins bons que dans le 2ème feature engineering.

Et avec le fichier de test :

In [130]:
print_result_CV_on_test_file(X_fe3_test, y_fe3_test, grid_search_fe3).head(30)

,MSE,RMSE,R2,MAE
0,5425869370696.760,2329349.560,0.630,1619483.700


In [131]:
print_coeffs_lasso(grid_search_fe3, X_fe3.columns)

                                    Feature  Coefficient
20                PropertyGFAOutsideParking  1804506.503
5                       PrimaryPropertyType   893665.261
2                        PropertyGFAParking   423121.980
11                  usage_Commerce & Retail   397103.427
17                     usage_Soins médicaux   386510.708
4                             YearBuilt_Bin   359143.480
15                       usage_Restauration   287842.925
14          usage_Loisirs et Divertissement   275333.635
8                             steam_percent   158480.842
9                              usage_Autres   156793.802
0                         NumberofBuildings   106325.893
10       usage_Bureaux & Espaces de travail    42086.222
1                            NumberofFloors    35289.447
3                              Neighborhood    17789.060
13             usage_Hébergement & Logement       -0.000
7                               gaz_percent        0.000
16  usage_Services publics & In

**Validation croisée avec le modèle GradientBoostingRegressor**

In [133]:
# Définition des hyperparamètres pour la recherche
param_grid = {
    'n_estimators': [50, 100, 150],  # Nombre d'arbres dans l'ensemble.
    'learning_rate': [0.01, 0.1, 0.2], # Taux d'apprentissage pour la réduction du poids de chaque arbre
    'max_depth': [3, 5, 7], # Profondeur maximale de chaque arbre
    'subsample': [0.8, 1.0] # Fraction des échantillons utilisés pour entraîner chaque arbre.
}

grid_search_fe3_gradient = fit_GridSearchCV_GradientBoostingRegressor(X_fe3_train, y_fe3_train, scoring, param_grid)
print_result_CV(grid_search_fe3_gradient)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Meilleurs paramètres : {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Meilleu(s) score sur le jeu d'entraînement:
0.6362409779884635
Résultats de la validation croisée :

Scores pour 'MAE':
MAE = -2163299.597 (+/-309384.149) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}
MAE = -2169020.075 (+/-303768.143) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50, 'subsample': 1.0}
MAE = -1862756.231 (+/-268436.406) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
MAE = -1868803.932 (+/-260129.944) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
MAE = -1704614.706 (+/-242357.460) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 150, 'subsample': 0.8}
MAE = -1716319.161 (+/-238007.450) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 150, 'subsample': 1.0}
MAE = -2086

Affichons les résultats dans un dataframe :

In [135]:
print_result_CV_as_dataframe(grid_search_fe3_gradient, scoring).head(30)

Résultats de la validation croisée :


,params_,mean_score_MAE,mean_score_R2,mean_score_RMSE,mean_fit_time
0,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",-1393917.924,0.636,-2102056.668,0.488
1,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",-1400802.251,0.633,-2112631.516,0.708
2,"{'learning_rate': 0.2, 'max_depth': 3, 'n_esti...",-1428710.290,0.628,-2130215.572,0.471
3,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",-1411790.334,0.627,-2127600.171,0.783
4,"{'learning_rate': 0.2, 'max_depth': 3, 'n_esti...",-1436323.366,0.627,-2132436.581,0.252
5,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",-1414814.454,0.627,-2130030.144,0.508
6,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",-1409328.957,0.627,-2129385.994,0.354
7,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",-1408897.249,0.624,-2138095.070,0.659
8,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",-1452116.187,0.621,-2146291.947,0.263
9,"{'learning_rate': 0.2, 'max_depth': 3, 'n_esti...",-1430197.257,0.617,-2155858.896,0.275


Les résultats sont très légèrement mieux que dans le 2ème feature engineering.

### 3.2 - 4ème feature Engineering

Je suis reparti du résultat du 2ème feature engineering qui avait obtenu les meilleurs résultats dans les prédictions.

Je vais utiliser ici QuantileTransformer pour normaliser des données qui seraient asymétriques (biaisée :

In [138]:
# Charger le fichier de données
data_fe4 = pd.read_csv("C:/Users/admin/Documents/Projets/Projet_4/data_projet/cleaned/2016_Building_Energy_Benchmarking_fe4.csv", sep=',', low_memory=False)
data_fe4.head()

,NumberofBuildings,NumberofFloors,PropertyGFAParking,SiteEnergyUseWN(kBtu),TotalGHGEmissions,Neighborhood_BALLARD,Neighborhood_CENTRAL,Neighborhood_DELRIDGE,Neighborhood_DELRIDGE NEIGHBORHOODS,Neighborhood_DOWNTOWN,Neighborhood_EAST,Neighborhood_GREATER DUWAMISH,Neighborhood_LAKE UNION,Neighborhood_MAGNOLIA / QUEEN ANNE,Neighborhood_NORTH,Neighborhood_NORTHEAST,Neighborhood_NORTHWEST,Neighborhood_SOUTHEAST,Neighborhood_SOUTHWEST,"YearBuilt_Bin_(1899.885, 1911.5]","YearBuilt_Bin_(1911.5, 1923.0]","YearBuilt_Bin_(1923.0, 1934.5]","YearBuilt_Bin_(1934.5, 1946.0]","YearBuilt_Bin_(1946.0, 1957.5]","YearBuilt_Bin_(1957.5, 1969.0]","YearBuilt_Bin_(1969.0, 1980.5]","YearBuilt_Bin_(1980.5, 1992.0]","YearBuilt_Bin_(1992.0, 2003.5]","YearBuilt_Bin_(2003.5, 2015.0]",PrimaryPropertyType_Distribution Center,PrimaryPropertyType_Hospital,PrimaryPropertyType_Hotel,PrimaryPropertyType_K-12 School,PrimaryPropertyType_Laboratory,PrimaryPropertyType_Large Office,PrimaryPropertyType_Low-Rise Multifamily,PrimaryPropertyType_Medical Office,PrimaryPropertyType_Mixed Use Property,PrimaryPropertyType_Office,PrimaryPropertyType_Other,PrimaryPropertyType_Refrigerated Warehouse,PrimaryPropertyType_Residence Hall,PrimaryPropertyType_Restaurant,PrimaryPropertyType_Retail Store,PrimaryPropertyType_Self-Storage Facility,PrimaryPropertyType_Senior Care Community,PrimaryPropertyType_Small- and Mid-Sized Office,PrimaryPropertyType_Supermarket / Grocery Store,PrimaryPropertyType_University,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship Facility,electricity_percent,gaz_percent,steam_percent,usage_Autres,usage_Bureaux & Espaces de travail,usage_Commerce & Retail,usage_Entrepôts et Logistique,usage_Hébergement & Logement,usage_Loisirs et Divertissement,usage_Restauration,usage_Services publics & Infrastructure,usage_Soins médicaux,usage_Transports & Parking,usage_Éducation,PropertyGFAOutsideParking
0,1.000,12,-5.199,7456910.000,249.980,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,54.610,17.660,27.730,0.000,0.000,0.000,0.000,100.000,0.000,0.000,0.000,0.000,0.000,0.000,0.874
1,1.000,11,1.191,8664479.000,295.860,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,38.660,61.340,0.000,0.000,0.000,0.000,0.000,80.990,0.000,4.460,0.000,0.000,14.550,0.000,0.877
2,1.000,10,-5.199,6946800.500,286.430,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,40.750,26.660,32.590,0.000,0.000,0.000,0.000,100.000,0.000,0.000,0.000,0.000,0.000,0.000,0.512
3,1.000,18,1.740,14656503.000,505.010,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,37.880,62.120,0.000,0.000,0.000,0.000,0.000,64.480,0.000,0.000,0.000,0.000,35.520,0.000,1.174
4,1.000,2,1.557,12581712.000,301.810,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,60.990,39.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,100.000,0.000,0.000,0.000,0.489


In [139]:
data_fe4.shape

(1440, 66)

**Sélectionner les features et la cible :**

In [141]:
y_fe4_conso = data_fe4['SiteEnergyUseWN(kBtu)']
X_fe4 = data_fe4.drop('SiteEnergyUseWN(kBtu)', axis=1, inplace=False)
X_fe4.shape

(1440, 65)

In [142]:
y_fe4_emissions = data_fe4['TotalGHGEmissions']
X_fe4 = X_fe4.drop('TotalGHGEmissions', axis=1, inplace=False)
X_fe4.shape

(1440, 64)

**Standardiser les valeurs et créer les jeux d'entraînement / test**

In [144]:
X_scale_fe4 = StandardScaler().fit_transform(X_fe4)

# 25% des données dans le jeu de test
X_fe4_train, X_fe4_test, y_fe4_train, y_fe4_test = model_selection.train_test_split(X_scale_fe4, y_fe4_conso, test_size=0.25, random_state=42 )

df = pd.DataFrame(X_scale_fe4)
df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63
count,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000
mean,0.000,0.000,-0.000,0.000,-0.000,-0.000,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,-0.000,-0.000,0.000,0.000,-0.000,-0.000,-0.000,-0.000,0.000,-0.000,-0.000,-0.000,0.000,0.000,0.000,0.000,-0.000,-0.000,0.000,0.000,-0.000,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,0.000,-0.000,-0.000,0.000,-0.000,0.000,0.000,-0.000,0.000,-0.000,0.000,0.000
std,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
min,-0.132,-0.820,-0.459,-0.216,-0.184,-0.175,-0.026,-0.481,-0.278,-0.542,-0.303,-0.315,-0.216,-0.288,-0.239,-0.175,-0.165,-0.341,-0.290,-0.324,-0.184,-0.350,-0.434,-0.361,-0.347,-0.355,-0.311,-0.192,-0.026,-0.209,-0.306,-0.053,-0.278,-0.037,-0.141,-0.269,-0.046,-0.415,-0.088,-0.119,-0.088,-0.247,-0.138,-0.103,-0.495,-0.165,-0.103,-0.385,-0.221,-2.599,-1.092,-0.202,-0.269,-0.761,-0.414,-0.523,-0.301,-0.159,-0.158,-0.296,-0.094,-0.367,-0.345,-5.128
25%,-0.132,-0.553,-0.459,-0.216,-0.184,-0.175,-0.026,-0.481,-0.278,-0.542,-0.303,-0.315,-0.216,-0.288,-0.239,-0.175,-0.165,-0.341,-0.290,-0.324,-0.184,-0.350,-0.434,-0.361,-0.347,-0.355,-0.311,-0.192,-0.026,-0.209,-0.306,-0.053,-0.278,-0.037,-0.141,-0.269,-0.046,-0.415,-0.088,-0.119,-0.088,-0.247,-0.138,-0.103,-0.495,-0.165,-0.103,-0.385,-0.221,-0.803,-1.092,-0.202,-0.269,-0.761,-0.414,-0.523,-0.301,-0.159,-0.158,-0.296,-0.094,-0.367,-0.345,-0.665
50%,-0.132,-0.286,-0.459,-0.216,-0.184,-0.175,-0.026,-0.481,-0.278,-0.542,-0.303,-0.315,-0.216,-0.288,-0.239,-0.175,-0.165,-0.341,-0.290,-0.324,-0.184,-0.350,-0.434,-0.361,-0.347,-0.355,-0.311,-0.192,-0.026,-0.209,-0.306,-0.053,-0.278,-0.037,-0.141,-0.269,-0.046,-0.415,-0.088,-0.119,-0.088,-0.247,-0.138,-0.103,-0.495,-0.165,-0.103,-0.385,-0.221,0.001,-0.084,-0.202,-0.269,-0.761,-0.414,-0.523,-0.301,-0.159,-0.158,-0.296,-0.094,-0.367,-0.345,0.000
75%,-0.132,0.247,-0.459,-0.216,-0.184,-0.175,-0.026,-0.481,-0.278,-0.542,-0.303,-0.315,-0.216,-0.288,-0.239,-0.175,-0.165,-0.341,-0.290,-0.324,-0.184,-0.350,-0.434,-0.361,-0.347,-0.355,-0.311,-0.192,-0.026,-0.209,-0.306,-0.053,-0.278,-0.037,-0.141,-0.269,-0.046,-0.415,-0.088,-0.119,-0.088,-0.247,-0.138,-0.103,-0.495,-0.165,-0.103,-0.385,-0.221,1.159,0.818,-0.202,-0.269,0.913,-0.414,-0.523,-0.301,-0.159,-0.158,-0.296,-0.094,-0.367,-0.345,0.665
max,12.515,25.594,3.622,4.637,5.444,5.700,37.934,2.077,3.603,1.845,3.302,3.174,4.637,3.477,4.179,5.700,6.074,2.933,3.443,3.084,5.444,2.859,2.306,2.770,2.879,2.819,3.215,5.219,37.934,4.796,3.272,18.947,3.603,26.814,7.101,3.721,21.886,2.407,11.398,8.426,11.398,4.043,7.234,9.747,2.022,6.074,9.747,2.597,4.527,1.159,2.652,9.046,4.509,1.834,3.037,2.199,3.859,8.574,10.992,3.805,11.920,6.989,2.988,5.127


**Tests de modèles sans validation croisée**

In [146]:
warnings.filterwarnings("ignore")

scores_array_fe4 = run_fit_models(X_fe4_train, y_fe4_train, X_fe4_test, y_fe4_test, X_fe4.columns)

df_results_fe4 = print_result_on_test_file(scores_array_fe4)
df_results_fe4.head(10)

Importance des features dans le RandomForestRegressor :
                                            Feature  Importance
63                        PropertyGFAOutsideParking       0.461
45  PrimaryPropertyType_Supermarket / Grocery Store       0.079
55                    usage_Entrepôts et Logistique       0.078
49                              electricity_percent       0.036
1                                    NumberofFloors       0.032
50                                      gaz_percent       0.028
2                                PropertyGFAParking       0.027
58                               usage_Restauration       0.027
60                             usage_Soins médicaux       0.024
53               usage_Bureaux & Espaces de travail       0.015
54                          usage_Commerce & Retail       0.013
57                  usage_Loisirs et Divertissement       0.013
52                                     usage_Autres       0.011
61                       usage_Transports & Park

,index,Modèle,MSE,RMSE,R2,MAE,ELAPSED_TIME
0,5,gradientBoostingRegressor,4478355354595.11,2116212.5,0.690,1417038.52,0.326
1,4,RamdomForestRegressor,4765297191884.85,2182956.07,0.670,1455636.7,1.228
2,1,ridge,5715437716047.72,2390698.16,0.610,1710656.07,0.003
3,2,lasso,5711939483597.89,2389966.42,0.610,1710238.74,0.062
4,3,elasticNet,6316360359551.36,2513237.03,0.560,1775991.84,0.003
5,0,DummyRegressor,14578031018368.1,3818118.78,-0.010,2862093.8,0.001
6,6,SVR,17300526148671.54,4159390.12,-0.190,2674588.81,0.123


**Validation croisée avec le modèle Lasso**

In [148]:
# Définition des hyperparamètres à tester
param_grid = {
    'alpha': np.logspace(-7, 7, 13) 
}

grid_search_fe4 = fit_GridSearchCV_lasso(X_fe4_train, y_fe4_train, scoring, param_grid)
print_result_CV(grid_search_fe4)

Fitting 5 folds for each of 13 candidates, totalling 65 fits
Meilleurs paramètres : {'alpha': 46415.888336127915}
Meilleu(s) score sur le jeu d'entraînement:
0.5480542327172078
Résultats de la validation croisée :

Scores pour 'MAE':
MAE = -1666193.050 (+/-193923.375) for {'alpha': 1e-07}
MAE = -1666193.050 (+/-193923.375) for {'alpha': 1.4677992676220705e-06}
MAE = -1666193.050 (+/-193923.375) for {'alpha': 2.1544346900318867e-05}
MAE = -1666193.050 (+/-193923.375) for {'alpha': 0.00031622776601683794}
MAE = -1666193.042 (+/-193923.379) for {'alpha': 0.004641588833612782}
MAE = -1666192.923 (+/-193923.427) for {'alpha': 0.06812920690579623}
MAE = -1666191.186 (+/-193924.134) for {'alpha': 1.0}
MAE = -1666164.929 (+/-193931.723) for {'alpha': 14.677992676220736}
MAE = -1665800.988 (+/-194050.784) for {'alpha': 215.44346900318865}
MAE = -1660958.197 (+/-195097.891) for {'alpha': 3162.2776601683795}
MAE = -1630353.366 (+/-195894.715) for {'alpha': 46415.888336127915}
MAE = -2073092.395 (

In [149]:
print_result_CV_as_dataframe(grid_search_fe4, scoring).head(30)

Résultats de la validation croisée :


,params_,mean_score_MAE,mean_score_R2,mean_score_RMSE,mean_fit_time
0,{'alpha': 46415.888336127915},-1630353.366,0.548,-2338737.808,0.000
1,{'alpha': 3162.2776601683795},-1660958.197,0.536,-2366795.414,0.006
2,{'alpha': 215.44346900318865},-1665800.988,0.534,-2371700.843,0.028
3,{'alpha': 14.677992676220736},-1666164.929,0.534,-2372079.726,0.034
4,{'alpha': 1.0},-1666191.186,0.534,-2372107.666,0.034
5,{'alpha': 0.06812920690579623},-1666192.923,0.534,-2372109.637,0.034
6,{'alpha': 0.004641588833612782},-1666193.042,0.534,-2372109.772,0.034
7,{'alpha': 0.00031622776601683794},-1666193.050,0.534,-2372109.781,0.031
8,{'alpha': 2.1544346900318867e-05},-1666193.050,0.534,-2372109.781,0.028
9,{'alpha': 1.4677992676220705e-06},-1666193.050,0.534,-2372109.781,0.035


Les scores sont moins bons que dans le 2ème feature engineering.

Et avec le fichier de test :

In [151]:
print_result_CV_on_test_file(X_fe4_test, y_fe4_test, grid_search_fe4).head(30)

,MSE,RMSE,R2,MAE
0,5536278276396.850,2352929.720,0.620,1659274.180


In [152]:
print_coeffs_lasso(grid_search_fe4, X_fe4.columns)

                                            Feature  Coefficient
63                        PropertyGFAOutsideParking  1741036.487
45  PrimaryPropertyType_Supermarket / Grocery Store   883334.042
32                 PrimaryPropertyType_Large Office   404908.989
2                                PropertyGFAParking   326423.884
58                               usage_Restauration   287372.895
31                   PrimaryPropertyType_Laboratory   285146.388
25                   YearBuilt_Bin_(1992.0, 2003.5]   249566.514
57                  usage_Loisirs et Divertissement   218181.142
37                        PrimaryPropertyType_Other   212960.604
43        PrimaryPropertyType_Senior Care Community   209404.828
34               PrimaryPropertyType_Medical Office   201349.936
1                                    NumberofFloors   201099.366
38       PrimaryPropertyType_Refrigerated Warehouse   151180.178
60                             usage_Soins médicaux   122411.213
51                       

**Validation croisée avec le modèle GradientBoostingRegressor**

In [154]:
# Définition des hyperparamètres pour la recherche
param_grid = {
    'n_estimators': [50, 100, 150],  # Nombre d'arbres dans l'ensemble.
    'learning_rate': [0.01, 0.1, 0.2], # Taux d'apprentissage pour la réduction du poids de chaque arbre
    'max_depth': [3, 5, 7], # Profondeur maximale de chaque arbre
    'subsample': [0.8, 1.0] # Fraction des échantillons utilisés pour entraîner chaque arbre.
}

grid_search_fe4_gradient = fit_GridSearchCV_GradientBoostingRegressor(X_fe4_train, y_fe4_train, scoring, param_grid)
print_result_CV(grid_search_fe4_gradient)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Meilleurs paramètres : {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 150, 'subsample': 0.8}
Meilleu(s) score sur le jeu d'entraînement:
0.6383134299841744
Résultats de la validation croisée :

Scores pour 'MAE':
MAE = -2173645.602 (+/-262000.908) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}
MAE = -2178254.993 (+/-269477.647) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50, 'subsample': 1.0}
MAE = -1898841.235 (+/-212894.970) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
MAE = -1910194.416 (+/-223261.471) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
MAE = -1750604.630 (+/-180309.826) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 150, 'subsample': 0.8}
MAE = -1770600.598 (+/-181187.335) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 150, 'subsample': 1.0}
MAE = -2112

Affichons les résultats dans un dataframe :

In [156]:
print_result_CV_as_dataframe(grid_search_fe4_gradient, scoring).head(30)

Résultats de la validation croisée :


,params_,mean_score_MAE,mean_score_R2,mean_score_RMSE,mean_fit_time
0,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",-1374803.082,0.638,-2095578.628,0.805
1,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",-1386228.860,0.637,-2099377.863,0.533
2,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",-1406828.426,0.632,-2115997.541,0.611
3,"{'learning_rate': 0.2, 'max_depth': 5, 'n_esti...",-1405645.595,0.630,-2120579.347,0.426
4,"{'learning_rate': 0.2, 'max_depth': 3, 'n_esti...",-1404512.803,0.630,-2120670.720,0.332
5,"{'learning_rate': 0.2, 'max_depth': 3, 'n_esti...",-1399204.987,0.630,-2119174.610,0.293
6,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",-1404043.132,0.630,-2121255.251,0.913
7,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",-1399335.999,0.628,-2128920.008,0.812
8,"{'learning_rate': 0.2, 'max_depth': 3, 'n_esti...",-1414178.286,0.627,-2131433.390,0.636
9,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",-1408706.085,0.626,-2133610.842,0.414


Les scores sont légèrement mieux que ceux du 3ème feature engineering pour le modèle GrandienBoostingRegressor, mais avec un temps de traitement bien plus long.

### 3.3 - 5ème feature Engineering

Je suis reparti du résultat du 2ème feature engineering qui avait obtenu les meilleurs résultats dans les prédictions. Et j'ai fait une réduction de dimensions avec l'ACP.

In [159]:
# Charger le fichier de données
data_fe5 = pd.read_csv("C:/Users/admin/Documents/Projets/Projet_4/data_projet/cleaned/2016_Building_Energy_Benchmarking_fe5.csv", sep=',', low_memory=False)
data_fe5.head()

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,PC21,PC22,PC23,PC24,PC25,PC26,PC27,PC28,PC29,PC30,PC31,PC32,PC33,PC34,PC35,SiteEnergyUseWN(kBtu),TotalGHGEmissions
0,1.453,-2.999,0.084,5.592,0.056,-1.762,-0.608,-1.891,-1.181,0.965,-0.307,0.318,0.558,-1.432,-0.736,-0.231,0.792,-0.242,0.370,-0.104,0.239,-0.641,0.048,-2.127,1.220,1.162,0.753,-1.007,-1.315,-0.586,-0.014,-0.439,-0.411,0.407,-0.181,7456910.000,249.980
1,1.217,-3.128,-1.121,4.585,0.934,-1.352,-0.559,-0.392,0.400,0.132,-1.735,-0.574,-0.717,-0.342,-0.749,-0.760,0.767,0.666,-0.356,-0.708,1.024,-0.883,-0.822,0.098,0.870,-0.209,0.032,-0.827,0.874,-1.363,0.294,-0.571,-0.499,-0.291,0.046,8664479.000,295.860
2,0.999,-3.167,0.357,5.657,-0.019,-1.749,-0.557,-2.010,-1.129,0.867,-0.348,0.061,0.529,-1.624,-0.749,-0.242,0.646,-0.369,0.260,-0.180,0.209,-0.497,0.174,-2.053,1.165,1.276,0.743,-1.002,-1.359,-0.539,0.076,-0.277,-0.377,0.486,-0.143,6946800.500,286.430
3,2.181,-2.703,-1.730,4.862,0.521,-0.385,-0.265,-0.112,-0.116,0.412,-1.562,-0.044,0.534,0.435,-2.015,-0.895,0.562,-0.227,0.226,-0.916,-0.128,-1.373,0.200,0.233,1.672,0.061,-0.822,-1.562,0.180,-2.028,-0.003,-1.268,-0.201,-0.636,0.801,14656503.000,505.010
4,-0.095,-1.621,0.475,0.174,0.933,2.158,0.570,0.741,-1.099,0.109,-0.849,0.097,-0.671,0.010,-0.668,-0.096,0.644,0.685,-0.268,0.194,1.136,-0.657,-0.489,0.523,-0.260,-0.494,0.630,0.279,1.034,0.224,0.594,-0.160,-0.380,-0.520,-1.144,12581712.000,301.810


In [160]:
data_fe5.shape

(1440, 37)

**Sélectionner les features et la cible :**

In [162]:
y_fe5_conso = data_fe5['SiteEnergyUseWN(kBtu)']
X_fe5 = data_fe5.drop('SiteEnergyUseWN(kBtu)', axis=1, inplace=False)
X_fe5.shape

(1440, 36)

In [163]:
y_fe5_emissions = data_fe5['TotalGHGEmissions']
X_fe5 = X_fe5.drop('TotalGHGEmissions', axis=1, inplace=False)
X_fe5.shape

(1440, 35)

**Standardiser les valeurs et créer les jeux d'entraînement / test**

In [165]:
X_scale_fe5 = StandardScaler().fit_transform(X_fe5)

df = pd.DataFrame(X_scale_fe5)
df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34
count,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000
mean,0.000,0.000,0.000,-0.000,-0.000,-0.000,0.000,-0.000,-0.000,-0.000,0.000,0.000,0.000,-0.000,-0.000,-0.000,0.000,0.000,-0.000,0.000,0.000,-0.000,-0.000,-0.000,-0.000,0.000,0.000,0.000,-0.000,-0.000,0.000,0.000,0.000,0.000,0.000
std,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
min,-1.953,-2.149,-3.645,-1.857,-1.992,-3.336,-2.173,-3.604,-3.773,-1.568,-2.922,-3.793,-3.978,-3.635,-4.874,-4.976,-3.655,-3.395,-5.311,-7.220,-5.475,-3.976,-5.258,-6.220,-5.855,-5.804,-4.086,-9.858,-10.359,-5.107,-2.790,-3.477,-3.676,-6.156,-5.847
25%,-0.746,-0.759,-0.615,-0.688,-0.665,-0.646,-0.653,-0.517,-0.531,-0.451,-0.650,-0.570,-0.667,-0.625,-0.555,-0.482,-0.536,-0.556,-0.586,-0.589,-0.500,-0.596,-0.443,-0.617,-0.517,-0.546,-0.638,-0.540,-0.495,-0.424,-0.585,-0.580,-0.420,-0.580,-0.488
50%,-0.188,-0.059,0.052,-0.106,-0.098,-0.095,-0.114,-0.011,-0.090,-0.039,-0.130,-0.001,-0.079,-0.142,0.076,0.023,0.014,-0.008,0.009,-0.060,-0.044,-0.037,0.050,0.032,-0.015,-0.064,-0.197,-0.034,0.111,0.012,0.005,-0.016,-0.106,-0.024,0.019
75%,0.705,0.593,0.682,0.499,0.393,0.452,0.331,0.354,0.451,0.252,0.518,0.550,0.531,0.537,0.568,0.636,0.535,0.557,0.597,0.514,0.499,0.509,0.665,0.562,0.551,0.529,0.506,0.460,0.534,0.520,0.546,0.412,0.362,0.533,0.521
max,3.243,2.323,3.990,4.024,3.563,3.552,4.520,8.064,6.374,16.654,4.499,4.193,4.450,4.796,8.219,5.330,8.112,6.221,12.028,8.872,10.197,5.195,5.257,7.004,7.053,11.978,4.537,10.643,17.062,7.354,10.739,6.324,10.934,7.578,8.297


In [166]:
# 25% des données dans le jeu de test
X_fe5_train, X_fe5_test, y_fe5_train, y_fe5_test = model_selection.train_test_split(X_scale_fe5, y_fe5_conso, test_size=0.25, random_state=42 )

**Tests de modèles sans validation croisée**

In [168]:
warnings.filterwarnings("ignore")

scores_array_fe5 = run_fit_models(X_fe5_train, y_fe5_train, X_fe5_test, y_fe5_test, X_fe5.columns)

Importance des features dans le RandomForestRegressor :
   Feature  Importance
0      PC1       0.209
1      PC2       0.095
2      PC3       0.073
4      PC5       0.048
8      PC9       0.041
10    PC11       0.038
9     PC10       0.030
7      PC8       0.026
6      PC7       0.025
15    PC16       0.024
17    PC18       0.022
24    PC25       0.022
3      PC4       0.022
32    PC33       0.022
30    PC31       0.020
5      PC6       0.020
14    PC15       0.019
22    PC23       0.019
21    PC22       0.017
33    PC34       0.017
18    PC19       0.016
16    PC17       0.016
23    PC24       0.015
26    PC27       0.014
27    PC28       0.013
29    PC30       0.013
19    PC20       0.013
25    PC26       0.012
13    PC14       0.012
34    PC35       0.012
12    PC13       0.012
20    PC21       0.012
31    PC32       0.011
11    PC12       0.011
28    PC29       0.010


In [169]:
df_results_fe5 = print_result_on_test_file(scores_array_fe5)
df_results_fe5.head(10)

,index,Modèle,MSE,RMSE,R2,MAE,ELAPSED_TIME
0,1,ridge,6298481694288.85,2509677.61,0.570,1787986.81,0.003
1,2,lasso,6299164382202.96,2509813.62,0.570,1787940.89,0.002
2,5,gradientBoostingRegressor,7154283899332.81,2674749.32,0.510,1914899.77,1.71
3,3,elasticNet,7310938216819.52,2703874.67,0.500,1961325.44,0.002
4,4,RamdomForestRegressor,7693662278526.76,2773745.17,0.470,1997516.7,4.747
5,0,DummyRegressor,14578031018368.1,3818118.78,-0.010,2862093.8,0.001
6,6,SVR,17300570937015.26,4159395.5,-0.190,2674591.24,0.115


**Validation croisée avec le modèle Lasso**

In [171]:
# Définition des hyperparamètres à tester
param_grid = {
    'alpha': np.logspace(-7, 7, 13) 
}

grid_search_fe5 = fit_GridSearchCV_lasso(X_fe5_train, y_fe5_train, scoring, param_grid)
print_result_CV(grid_search_fe5)

Fitting 5 folds for each of 13 candidates, totalling 65 fits
Meilleurs paramètres : {'alpha': 46415.888336127915}
Meilleu(s) score sur le jeu d'entraînement:
0.4816679800127449
Résultats de la validation croisée :

Scores pour 'MAE':
MAE = -1797029.543 (+/-231127.084) for {'alpha': 1e-07}
MAE = -1797029.543 (+/-231127.084) for {'alpha': 1.4677992676220705e-06}
MAE = -1797029.543 (+/-231127.084) for {'alpha': 2.1544346900318867e-05}
MAE = -1797029.543 (+/-231127.084) for {'alpha': 0.00031622776601683794}
MAE = -1797029.543 (+/-231127.080) for {'alpha': 0.004641588833612782}
MAE = -1797029.533 (+/-231127.028) for {'alpha': 0.06812920690579623}
MAE = -1797029.384 (+/-231126.262) for {'alpha': 1.0}
MAE = -1797027.193 (+/-231114.990) for {'alpha': 14.677992676220736}
MAE = -1796995.235 (+/-230949.786) for {'alpha': 215.44346900318865}
MAE = -1796665.600 (+/-228175.381) for {'alpha': 3162.2776601683795}
MAE = -1800891.292 (+/-196255.145) for {'alpha': 46415.888336127915}
MAE = -2465141.814 (

In [172]:
print_result_CV_as_dataframe(grid_search_fe5, scoring).head(30)

Résultats de la validation croisée :


,params_,mean_score_MAE,mean_score_R2,mean_score_RMSE,mean_fit_time
0,{'alpha': 46415.888336127915},-1800891.292,0.482,-2505339.880,0.000
1,{'alpha': 3162.2776601683795},-1796665.600,0.469,-2526326.412,0.001
2,{'alpha': 215.44346900318865},-1796995.235,0.467,-2528899.136,0.001
3,{'alpha': 14.677992676220736},-1797027.193,0.467,-2529083.599,0.001
4,{'alpha': 1.0},-1797029.384,0.467,-2529096.218,0.000
5,{'alpha': 0.06812920690579623},-1797029.533,0.467,-2529097.077,0.003
6,{'alpha': 0.004641588833612782},-1797029.543,0.467,-2529097.136,0.000
7,{'alpha': 0.00031622776601683794},-1797029.543,0.467,-2529097.140,0.000
8,{'alpha': 2.1544346900318867e-05},-1797029.543,0.467,-2529097.140,0.003
9,{'alpha': 1.4677992676220705e-06},-1797029.543,0.467,-2529097.140,0.003


Et avec le fichier de test :

In [174]:
print_result_CV_on_test_file(X_fe5_test, y_fe5_test, grid_search_fe5).head(30)

,MSE,RMSE,R2,MAE
0,6357716291485.460,2521451.230,0.560,1808843.800


In [175]:
print_coeffs_lasso(grid_search_fe5, X_fe5.columns)

   Feature  Coefficient
0      PC1  1270802.646
4      PC5   753977.547
21    PC22   582389.343
9     PC10   555092.290
6      PC7   552378.096
7      PC8   349008.950
23    PC24   336212.316
26    PC27   287812.088
25    PC26   204748.950
12    PC13   192864.007
24    PC25   144847.343
27    PC28   139273.964
3      PC4   138002.173
16    PC17   133020.418
30    PC31   122550.929
31    PC32    96712.154
33    PC34    69270.471
22    PC23    66762.572
29    PC30    10057.302
28    PC29       -0.000
34    PC35        0.000
13    PC14       -0.000
11    PC12       -0.000
14    PC15   -56688.842
32    PC33   -80040.201
20    PC21   -99299.683
5      PC6  -119693.984
18    PC19  -129478.155
15    PC16  -154586.478
19    PC20  -269426.929
8      PC9  -270628.248
17    PC18  -468391.765
10    PC11  -573223.863
2      PC3  -844136.630
1      PC2  -941306.931


**Validation croisée avec le modèle GradientBoostingRegressor**

In [177]:
# Définition des hyperparamètres pour la recherche
param_grid = {
    'n_estimators': [50, 100, 150],  # Nombre d'arbres dans l'ensemble.
    'learning_rate': [0.01, 0.1, 0.2], # Taux d'apprentissage pour la réduction du poids de chaque arbre
    'max_depth': [3, 5, 7], # Profondeur maximale de chaque arbre
    'subsample': [0.8, 1.0] # Fraction des échantillons utilisés pour entraîner chaque arbre.
}

grid_search_fe5_gradient = fit_GridSearchCV_GradientBoostingRegressor(X_fe5_train, y_fe5_train, scoring, param_grid)
print_result_CV(grid_search_fe5_gradient)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Meilleurs paramètres : {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 100, 'subsample': 0.8}
Meilleu(s) score sur le jeu d'entraînement:
0.49345506224667357
Résultats de la validation croisée :

Scores pour 'MAE':
MAE = -2440928.963 (+/-317310.616) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50, 'subsample': 0.8}
MAE = -2453292.208 (+/-316830.493) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 50, 'subsample': 1.0}
MAE = -2266596.291 (+/-284845.769) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
MAE = -2278412.905 (+/-294687.488) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
MAE = -2138090.781 (+/-252650.420) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 150, 'subsample': 0.8}
MAE = -2145287.524 (+/-254235.275) for {'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 150, 'subsample': 1.0}
MAE = -234

Affichons les résultats dans un dataframe :

In [179]:
print_result_CV_as_dataframe(grid_search_fe5_gradient, scoring).head(30)

Résultats de la validation croisée :


,params_,mean_score_MAE,mean_score_R2,mean_score_RMSE,mean_fit_time
0,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",-1759110.523,0.493,-2479104.599,5.376
1,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",-1758712.511,0.493,-2479750.512,7.943
2,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",-1765615.047,0.491,-2484927.939,2.696
3,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",-1751633.230,0.479,-2512087.108,7.435
4,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",-1752813.024,0.479,-2513110.992,4.727
5,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",-1772221.850,0.478,-2512669.004,5.839
6,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",-1779212.324,0.476,-2521772.012,2.377
7,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",-1776198.205,0.475,-2522330.936,3.757
8,"{'learning_rate': 0.2, 'max_depth': 5, 'n_esti...",-1782038.510,0.470,-2539759.036,7.110
9,"{'learning_rate': 0.2, 'max_depth': 5, 'n_esti...",-1782282.392,0.469,-2541166.614,4.775


Les résultats sont moins que ceux des précédents features engineering.

**Choix final - la suite se fera avec le dataset issu du 2ème feature engineering**

Lers 2ème, 3ème et 4ème avaient des résultats proches, mais le 2ème présente l'avantage d'être réutilisable pour l'autre cible, sans avoir à redéfnir la cilbe dans le TargetEncoder, et le 4ème était plus lent. 

### 4 - Simulation d'autres modèles et choix du modèle final

On repart du dataset issu du 2ème feature engineering

In [183]:
# Charger le fichier de données
data_fe2 = pd.read_csv("C:/Users/admin/Documents/Projets/Projet_4/data_projet/cleaned/2016_Building_Energy_Benchmarking_fe2.csv", sep=',', low_memory=False)
data_fe2.head()

,NumberofBuildings,NumberofFloors,PropertyGFAParking,SiteEnergyUseWN(kBtu),TotalGHGEmissions,Neighborhood_BALLARD,Neighborhood_CENTRAL,Neighborhood_DELRIDGE,Neighborhood_DELRIDGE NEIGHBORHOODS,Neighborhood_DOWNTOWN,Neighborhood_EAST,Neighborhood_GREATER DUWAMISH,Neighborhood_LAKE UNION,Neighborhood_MAGNOLIA / QUEEN ANNE,Neighborhood_NORTH,Neighborhood_NORTHEAST,Neighborhood_NORTHWEST,Neighborhood_SOUTHEAST,Neighborhood_SOUTHWEST,"YearBuilt_Bin_(1899.885, 1911.5]","YearBuilt_Bin_(1911.5, 1923.0]","YearBuilt_Bin_(1923.0, 1934.5]","YearBuilt_Bin_(1934.5, 1946.0]","YearBuilt_Bin_(1946.0, 1957.5]","YearBuilt_Bin_(1957.5, 1969.0]","YearBuilt_Bin_(1969.0, 1980.5]","YearBuilt_Bin_(1980.5, 1992.0]","YearBuilt_Bin_(1992.0, 2003.5]","YearBuilt_Bin_(2003.5, 2015.0]",PrimaryPropertyType_Distribution Center,PrimaryPropertyType_Hospital,PrimaryPropertyType_Hotel,PrimaryPropertyType_K-12 School,PrimaryPropertyType_Laboratory,PrimaryPropertyType_Large Office,PrimaryPropertyType_Low-Rise Multifamily,PrimaryPropertyType_Medical Office,PrimaryPropertyType_Mixed Use Property,PrimaryPropertyType_Office,PrimaryPropertyType_Other,PrimaryPropertyType_Refrigerated Warehouse,PrimaryPropertyType_Residence Hall,PrimaryPropertyType_Restaurant,PrimaryPropertyType_Retail Store,PrimaryPropertyType_Self-Storage Facility,PrimaryPropertyType_Senior Care Community,PrimaryPropertyType_Small- and Mid-Sized Office,PrimaryPropertyType_Supermarket / Grocery Store,PrimaryPropertyType_University,PrimaryPropertyType_Warehouse,PrimaryPropertyType_Worship Facility,electricity_percent,gaz_percent,steam_percent,usage_Autres,usage_Bureaux & Espaces de travail,usage_Commerce & Retail,usage_Entrepôts et Logistique,usage_Hébergement & Logement,usage_Loisirs et Divertissement,usage_Restauration,usage_Services publics & Infrastructure,usage_Soins médicaux,usage_Transports & Parking,usage_Éducation,PropertyGFAOutsideParking
0,1.000,12,0,7456910.000,249.980,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,54.610,17.660,27.730,0.000,0.000,0.000,0.000,100.000,0.000,0.000,0.000,0.000,0.000,0.000,88434.000
1,1.000,11,15064,8664479.000,295.860,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,38.660,61.340,0.000,0.000,0.000,0.000,0.000,80.990,0.000,4.460,0.000,0.000,14.550,0.000,88502.000
2,1.000,10,0,6946800.500,286.430,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,40.750,26.660,32.590,0.000,0.000,0.000,0.000,100.000,0.000,0.000,0.000,0.000,0.000,0.000,61320.000
3,1.000,18,62000,14656503.000,505.010,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,37.880,62.120,0.000,0.000,0.000,0.000,0.000,64.480,0.000,0.000,0.000,0.000,35.520,0.000,113580.000
4,1.000,2,37198,12581712.000,301.810,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,60.990,39.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,100.000,0.000,0.000,0.000,60090.000


**Préparation**

In [185]:
y_fe2_conso = data_fe2['SiteEnergyUseWN(kBtu)']
X_fe2 = data_fe2.drop('SiteEnergyUseWN(kBtu)', axis=1, inplace=False)
X_fe2.shape

(1440, 65)

In [186]:
y_fe2_emissions = data_fe2['TotalGHGEmissions']
X_fe2 = X_fe2.drop('TotalGHGEmissions', axis=1, inplace=False)
X_fe2.shape

(1440, 64)

In [187]:
X_scale_fe2 = StandardScaler().fit_transform(X_fe2)

df = pd.DataFrame(X_scale_fe2)
df.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63
count,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000,1440.000
mean,0.000,0.000,0.000,0.000,-0.000,-0.000,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,-0.000,-0.000,0.000,0.000,-0.000,-0.000,-0.000,-0.000,0.000,-0.000,-0.000,-0.000,0.000,0.000,0.000,0.000,-0.000,-0.000,0.000,0.000,-0.000,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,0.000,-0.000,-0.000,0.000,-0.000,0.000,0.000,-0.000,0.000,-0.000,0.000,-0.000
std,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000
min,-0.132,-0.820,-0.288,-0.216,-0.184,-0.175,-0.026,-0.481,-0.278,-0.542,-0.303,-0.315,-0.216,-0.288,-0.239,-0.175,-0.165,-0.341,-0.290,-0.324,-0.184,-0.350,-0.434,-0.361,-0.347,-0.355,-0.311,-0.192,-0.026,-0.209,-0.306,-0.053,-0.278,-0.037,-0.141,-0.269,-0.046,-0.415,-0.088,-0.119,-0.088,-0.247,-0.138,-0.103,-0.495,-0.165,-0.103,-0.385,-0.221,-2.599,-1.092,-0.202,-0.269,-0.761,-0.414,-0.523,-0.301,-0.159,-0.158,-0.296,-0.094,-0.367,-0.345,-1.049
25%,-0.132,-0.553,-0.288,-0.216,-0.184,-0.175,-0.026,-0.481,-0.278,-0.542,-0.303,-0.315,-0.216,-0.288,-0.239,-0.175,-0.165,-0.341,-0.290,-0.324,-0.184,-0.350,-0.434,-0.361,-0.347,-0.355,-0.311,-0.192,-0.026,-0.209,-0.306,-0.053,-0.278,-0.037,-0.141,-0.269,-0.046,-0.415,-0.088,-0.119,-0.088,-0.247,-0.138,-0.103,-0.495,-0.165,-0.103,-0.385,-0.221,-0.803,-1.092,-0.202,-0.269,-0.761,-0.414,-0.523,-0.301,-0.159,-0.158,-0.296,-0.094,-0.367,-0.345,-0.622
50%,-0.132,-0.286,-0.288,-0.216,-0.184,-0.175,-0.026,-0.481,-0.278,-0.542,-0.303,-0.315,-0.216,-0.288,-0.239,-0.175,-0.165,-0.341,-0.290,-0.324,-0.184,-0.350,-0.434,-0.361,-0.347,-0.355,-0.311,-0.192,-0.026,-0.209,-0.306,-0.053,-0.278,-0.037,-0.141,-0.269,-0.046,-0.415,-0.088,-0.119,-0.088,-0.247,-0.138,-0.103,-0.495,-0.165,-0.103,-0.385,-0.221,0.001,-0.084,-0.202,-0.269,-0.761,-0.414,-0.523,-0.301,-0.159,-0.158,-0.296,-0.094,-0.367,-0.345,-0.357
75%,-0.132,0.247,-0.288,-0.216,-0.184,-0.175,-0.026,-0.481,-0.278,-0.542,-0.303,-0.315,-0.216,-0.288,-0.239,-0.175,-0.165,-0.341,-0.290,-0.324,-0.184,-0.350,-0.434,-0.361,-0.347,-0.355,-0.311,-0.192,-0.026,-0.209,-0.306,-0.053,-0.278,-0.037,-0.141,-0.269,-0.046,-0.415,-0.088,-0.119,-0.088,-0.247,-0.138,-0.103,-0.495,-0.165,-0.103,-0.385,-0.221,1.159,0.818,-0.202,-0.269,0.913,-0.414,-0.523,-0.301,-0.159,-0.158,-0.296,-0.094,-0.367,-0.345,0.178
max,12.515,25.594,12.040,4.637,5.444,5.700,37.934,2.077,3.603,1.845,3.302,3.174,4.637,3.477,4.179,5.700,6.074,2.933,3.443,3.084,5.444,2.859,2.306,2.770,2.879,2.819,3.215,5.219,37.934,4.796,3.272,18.947,3.603,26.814,7.101,3.721,21.886,2.407,11.398,8.426,11.398,4.043,7.234,9.747,2.022,6.074,9.747,2.597,4.527,1.159,2.652,9.046,4.509,1.834,3.037,2.199,3.859,8.574,10.992,3.805,11.920,6.989,2.988,5.912


In [188]:
# 25% des données dans le jeu de test
X_fe2_train, X_fe2_test, y_fe2_train, y_fe2_test = model_selection.train_test_split(X_scale_fe2, y_fe2_conso, test_size=0.25, random_state=42 )

#### 4-1 - modèle Lasso

In [190]:
# Définition des hyperparamètres à tester
param_grid = {
    'alpha': np.logspace(-6, 6, 13) 
}

In [191]:
grid_search_fe2_lasso = fit_GridSearchCV_lasso(X_fe2_train, y_fe2_train, scoring, param_grid)
print_result_CV_as_dataframe(grid_search_fe2_lasso, scoring).head(30)

Fitting 5 folds for each of 13 candidates, totalling 65 fits
Résultats de la validation croisée :


,params_,mean_score_MAE,mean_score_R2,mean_score_RMSE,mean_fit_time
0,{'alpha': 10000.0},-1471929.332,0.616,-2162357.552,0.000
1,{'alpha': 1000.0},-1477961.342,0.614,-2168548.666,0.013
2,{'alpha': 100.0},-1478859.177,0.613,-2169500.190,0.031
3,{'alpha': 10.0},-1478863.803,0.613,-2169569.741,0.037
4,{'alpha': 1.0},-1478862.143,0.613,-2169578.331,0.031
5,{'alpha': 0.1},-1478861.631,0.613,-2169578.941,0.034
6,{'alpha': 0.01},-1478861.581,0.613,-2169579.005,0.034
7,{'alpha': 0.001},-1478861.576,0.613,-2169579.011,0.032
8,{'alpha': 0.0001},-1478861.576,0.613,-2169579.012,0.034
9,{'alpha': 1e-05},-1478861.576,0.613,-2169579.012,0.043


Sur le fichier de test :

In [193]:
print_result_CV_on_test_file(X_fe2_test, y_fe2_test, grid_search_fe2_lasso).head(30)

,MSE,RMSE,R2,MAE
0,5003749155805.970,2236906.160,0.650,1532310.570


#### 4-2 - modèle ElasticNet

In [195]:
param_grid = {
    'alpha': [0.01, 0.1, 1.0, 10.0, 100.0],  # Grille pour alpha
    'l1_ratio': [0.1, 0.5, 0.7, 0.9, 1.0]  # Grille pour l1_ratio
}

In [196]:
grid_search_fe2_elasticNet = fit_GridSearchCV_elasticNet(X_fe2_train, y_fe2_train, scoring, param_grid)
print_result_CV_as_dataframe(grid_search_fe2_elasticNet, scoring).head(30)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Résultats de la validation croisée :


,params_,mean_score_MAE,mean_score_R2,mean_score_RMSE,mean_fit_time
0,"{'alpha': 0.1, 'l1_ratio': 0.7}",-1479879.167,0.615,-2165137.733,0.009
1,"{'alpha': 0.1, 'l1_ratio': 0.5}",-1482556.165,0.615,-2165444.169,0.006
2,"{'alpha': 0.1, 'l1_ratio': 0.9}",-1478157.123,0.614,-2167178.702,0.025
3,"{'alpha': 0.01, 'l1_ratio': 0.1}",-1478161.309,0.614,-2167372.900,0.034
4,"{'alpha': 0.01, 'l1_ratio': 0.5}",-1478456.758,0.614,-2168271.828,0.034
5,"{'alpha': 0.01, 'l1_ratio': 0.7}",-1478664.926,0.614,-2168804.386,0.034
6,"{'alpha': 0.01, 'l1_ratio': 0.9}",-1478913.554,0.613,-2169381.680,0.028
7,"{'alpha': 100.0, 'l1_ratio': 1.0}",-1478859.177,0.613,-2169500.190,0.031
8,"{'alpha': 10.0, 'l1_ratio': 1.0}",-1478863.803,0.613,-2169569.741,0.037
9,"{'alpha': 1.0, 'l1_ratio': 1.0}",-1478862.143,0.613,-2169578.331,0.041


Résultat sur le fichier de test avec les meilleurs hyperparamètres :

In [198]:
print_result_CV_on_test_file(X_fe2_test, y_fe2_test, grid_search_fe2_elasticNet).head(30)

,MSE,RMSE,R2,MAE
0,5039515552295.320,2244886.530,0.650,1549455.890


#### 4.3 - modèle GradientBoostingRegressor

In [200]:
# Définition des hyperparamètres pour la recherche
param_grid = {
    'n_estimators': [50, 100, 150],  # Nombre d'arbres dans l'ensemble.
    'learning_rate': [0.01, 0.1, 0.2], # Taux d'apprentissage pour la réduction du poids de chaque arbre
    'max_depth': [3, 5, 7], # Profondeur maximale de chaque arbre
    'subsample': [0.8, 1.0] # Fraction des échantillons utilisés pour entraîner chaque arbre.
}

In [201]:
grid_search_fe2_gradient = fit_GridSearchCV_GradientBoostingRegressor(X_fe2_train, y_fe2_train, scoring, param_grid)
print_result_CV_as_dataframe(grid_search_fe2_gradient, scoring).head(30)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Résultats de la validation croisée :


,params_,mean_score_MAE,mean_score_R2,mean_score_RMSE,mean_fit_time
0,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",-1389995.643,0.634,-2106896.086,0.540
1,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",-1391892.298,0.632,-2113880.335,0.809
2,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",-1405946.893,0.632,-2115455.635,0.612
3,"{'learning_rate': 0.2, 'max_depth': 3, 'n_esti...",-1406688.937,0.631,-2116205.139,0.288
4,"{'learning_rate': 0.2, 'max_depth': 3, 'n_esti...",-1401008.154,0.631,-2118600.975,0.344
5,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",-1382381.030,0.631,-2120243.692,0.822
6,"{'learning_rate': 0.1, 'max_depth': 5, 'n_esti...",-1395471.969,0.630,-2121878.966,0.411
7,"{'learning_rate': 0.1, 'max_depth': 3, 'n_esti...",-1403296.954,0.630,-2120790.799,0.908
8,"{'learning_rate': 0.2, 'max_depth': 3, 'n_esti...",-1411471.738,0.627,-2129571.658,0.627
9,"{'learning_rate': 0.2, 'max_depth': 3, 'n_esti...",-1414874.430,0.624,-2133977.233,0.596


Les R2 sont meilleurs avec le GradientBoosting que le Lasso et l'ElasticNet, mais le temps de traitement est plus long.

Résultat sur le fichier de test avec les meilleurs hyperparamètres :

In [203]:
print_result_CV_on_test_file(X_fe2_test, y_fe2_test, grid_search_fe2_gradient).head(30)

,MSE,RMSE,R2,MAE
0,4605739041357.600,2146098.560,0.680,1418104.970


#### 4.4 - Modèle RamdomForestRegressor

In [205]:
param_grid = {
    'n_estimators': [100, 200, 300],  # nombre d'arbres dans la forêt.
    'max_depth': [None, 10, 20, 30],   # profondeur maximale des arbres
    'min_samples_split': [2, 5, 10],   # nombre minimum d'échantillons requis pour diviser un nœud interne
    'min_samples_leaf': [1, 2, 4],    # nombre minimum d'échantillons requis pour être une feuille
    'max_features': ['auto', 'sqrt', 'log2']   # nombre de caractéristiques considérées pour diviser à chaque nœud
}

In [206]:
grid_search_fe2_ramdomForest = fit_GridSearchCV_ramdomForestRegressor(X_fe2_train, y_fe2_train, scoring, param_grid)
print_result_CV_as_dataframe(grid_search_fe2_ramdomForest, scoring).head(30)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits
Résultats de la validation croisée :


,params_,mean_score_MAE,mean_score_R2,mean_score_RMSE,mean_fit_time
0,"{'max_depth': None, 'max_features': 'auto', 'm...",-1417555.461,0.618,-2154206.651,5.913
1,"{'max_depth': 30, 'max_features': 'auto', 'min...",-1417539.939,0.618,-2154475.340,6.428
2,"{'max_depth': 20, 'max_features': 'auto', 'min...",-1418746.730,0.618,-2154653.300,5.982
3,"{'max_depth': None, 'max_features': 'auto', 'm...",-1416448.263,0.618,-2154445.029,3.701
4,"{'max_depth': 30, 'max_features': 'auto', 'min...",-1416529.567,0.618,-2154818.093,4.060
5,"{'max_depth': None, 'max_features': 'auto', 'm...",-1416956.134,0.618,-2156177.506,1.775
6,"{'max_depth': None, 'max_features': 'auto', 'm...",-1418448.574,0.618,-2155891.679,5.939
7,"{'max_depth': 30, 'max_features': 'auto', 'min...",-1418605.084,0.618,-2155963.920,5.969
8,"{'max_depth': 20, 'max_features': 'auto', 'min...",-1419769.296,0.617,-2156367.581,5.791
9,"{'max_depth': 30, 'max_features': 'auto', 'min...",-1417190.708,0.617,-2156925.005,2.026


Ce modèle est moins performant que le GradientBoostingRegressor. Et les temps sont beaucoup trop élevés.

Résultat sur le Résultat sur le fichier de test avec les meilleurs hyperparamètres :

In [213]:
print_result_CV_on_test_file(X_fe2_test, y_fe2_test, grid_search_fe2_ramdomForest).head(30)

,MSE,RMSE,R2,MAE
0,4585488672668.730,2141375.420,0.680,1426744.450


#### 4.5 - Modèle SVR

In [215]:
param_grid = {
    'C': [0.1, 1, 10, 100],              # Le paramètre de régularisation (contrôle la marge entre biais et variance).
    'epsilon': [0.1, 0.2, 0.5, 1],       # La largeur de la zone d'insensibilité à l'erreur
    'kernel': ['rbf', 'poly', 'linear'], # Le type de noyau utilisé dans le modèle ('linear', 'poly', 'rbf', 'sigmoid')
}

In [217]:
grid_search_fe2_SVR = fit_GridSearchCV_SVR(X_fe2_train, y_fe2_train, scoring, param_grid)
print_result_CV_as_dataframe(grid_search_fe2_SVR, scoring).head(30)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Résultats de la validation croisée :


,params_,mean_score_MAE,mean_score_R2,mean_score_RMSE,mean_fit_time
0,"{'C': 100, 'epsilon': 1, 'kernel': 'linear'}",-2392086.977,-0.105,-3683579.521,0.122
1,"{'C': 100, 'epsilon': 0.5, 'kernel': 'linear'}",-2392086.991,-0.105,-3683579.553,0.112
2,"{'C': 100, 'epsilon': 0.2, 'kernel': 'linear'}",-2392086.998,-0.105,-3683579.572,0.112
3,"{'C': 100, 'epsilon': 0.1, 'kernel': 'linear'}",-2392087.001,-0.105,-3683579.579,0.109
4,"{'C': 10, 'epsilon': 0.1, 'kernel': 'linear'}",-2450626.791,-0.149,-3757047.480,0.116
5,"{'C': 10, 'epsilon': 0.2, 'kernel': 'linear'}",-2450626.791,-0.149,-3757047.480,0.119
6,"{'C': 10, 'epsilon': 0.5, 'kernel': 'linear'}",-2450626.792,-0.149,-3757047.482,0.112
7,"{'C': 10, 'epsilon': 1, 'kernel': 'linear'}",-2450626.792,-0.149,-3757047.485,0.116
8,"{'C': 100, 'epsilon': 0.1, 'kernel': 'poly'}",-2456931.320,-0.155,-3765879.165,0.119
9,"{'C': 100, 'epsilon': 0.2, 'kernel': 'poly'}",-2456931.320,-0.155,-3765879.180,0.115


Les résultats sont très mauvais avec SVR.

**Le modèle final choisi est : GrandientBoostingRegressor**

Le meilleur score R2 en validation croisée pour prédire la cible SiteEnergyUseWN(kBtu) est de 0.656

### 5 - Analyse de la "feature importance" globale et locale